# 소설 작가 분류

최종 모델 스코어
- logloss =   0.3904
- accuracy =  85.9163
- mean logloss =  0.3903828351640911

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [31]:
from matplotlib import rcParams, pyplot as plt
import numpy as np
import pandas as pd
import random as rn
import re
from pathlib import Path
import warnings
import gc
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.backend import clear_session
from tensorflow.keras import Input, Model, Sequential, layers
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import log_loss, accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
import lightgbm as lgbm
from sklearn.decomposition import IncrementalPCA

In [3]:
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

# 학습데이터 로드

In [14]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 2020 

In [13]:
feature_name = 'feature'
feature_target_file = feature_dir / f'{feature_name}_target.csv'

In [5]:
trn = pd.read_csv(trn_file, index_col=0)
print(trn.shape)
trn.head()

(54879, 2)


,text,author
index,,
0,"He was almost choking. There was so much, so m...",3
1,"“Your sister asked for it, I suppose?”",2
2,"She was engaged one day as she walked, in per...",1
3,"The captain was in the porch, keeping himself ...",4
4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [6]:
tst = pd.read_csv(tst_file, index_col=0)
print(tst.shape)
tst.head()

(19617, 1)


,text
index,
0,“Not at all. I think she is one of the most ch...
1,"""No,"" replied he, with sudden consciousness, ""..."
2,As the lady had stated her intention of scream...
3,“And then suddenly in the silence I heard a so...
4,His conviction remained unchanged. So far as I...


In [7]:
df_train = trn.copy()

# 실제 타겟 값 생성

In [15]:
y = df_train.loc[:, target_col]
y.to_csv(feature_target_file)

# Stacking Feature

## 1. CNN

### 1-1. bow

In [5]:
algo_name = 'cnn'
feature_name = 'bow'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'

p_val_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'

p_val_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
p_tst_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'

p_val_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
p_tst_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'

p_val_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'
p_tst_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'

p_val_ver5_file = val_dir / f'{model_name}_oof_pred_ver5.csv'
p_tst_ver5_file = tst_dir / f'{model_name}_test_pred_ver5.csv'

p_val_ver6_file = val_dir / f'{model_name}_oof_pred_ver6.csv'
p_tst_ver6_file = tst_dir / f'{model_name}_test_pred_ver6.csv'

sub_ver1_file = sub_dir / f'{model_name}_ver1.csv'
sub_ver2_file = sub_dir / f'{model_name}_ver2.csv'
sub_ver3_file = sub_dir / f'{model_name}_ver3.csv'
sub_ver4_file = sub_dir / f'{model_name}_ver4.csv'
sub_ver5_file = sub_dir / f'{model_name}_ver5.csv'
sub_ver6_file = sub_dir / f'{model_name}_ver6.csv'

#### DTM 피쳐 생성

- nltk의 word_tokenize 사용

In [27]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_1 = vec.fit_transform(trn['text']).toarray()
X_tst_1 = vec.transform(tst['text']).toarray()
print(X_1.shape, X_tst_1.shape)

(54879, 2685) (19617, 2685)


In [28]:
X_1[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

- nltk의 WordPunctTokenizer 사용

In [29]:
vec = CountVectorizer(tokenizer=wordPunctTokenizer.tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_2 = vec.fit_transform(trn['text']).toarray()
X_tst_2 = vec.transform(tst['text']).toarray()
print(X_2.shape, X_tst_2.shape)

(54879, 2655) (19617, 2655)


In [30]:
X_2[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 4, 0, 0, 0])

- keras의 text_to_word_sequence 사용

In [31]:
vec = CountVectorizer(tokenizer=text_to_word_sequence, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_3 = vec.fit_transform(trn['text']).toarray()
X_tst_3 = vec.transform(tst['text']).toarray()
print(X_3.shape, X_tst_3.shape)

(54879, 1907) (19617, 1907)


In [32]:
X_3[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

- nltk의 word_tokenize 사용 , stopword 없음

In [33]:
vec = CountVectorizer(tokenizer=word_tokenize, ngram_range=(1, 2), min_df=100)
X_4 = vec.fit_transform(trn['text']).toarray()
X_tst_4 = vec.transform(tst['text']).toarray()
print(X_4.shape, X_tst_4.shape)

(54879, 4720) (19617, 4720)


In [34]:
X_4[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

- nltk의 WordPunctTokenizer 사용 , stopword 없음

In [35]:
vec = CountVectorizer(tokenizer=wordPunctTokenizer.tokenize, ngram_range=(1, 2), min_df=100)
X_5 = vec.fit_transform(trn['text']).toarray()
X_tst_5 = vec.transform(tst['text']).toarray()
print(X_5.shape, X_tst_5.shape)

(54879, 4777) (19617, 4777)


In [36]:
X_5[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

- keras의 text_to_word_sequence 사용 , stopword 없음

In [37]:
vec = CountVectorizer(tokenizer=text_to_word_sequence, ngram_range=(1, 2), min_df=100)
X_6 = vec.fit_transform(trn['text']).toarray()
X_tst_6 = vec.transform(tst['text']).toarray()
print(X_6.shape, X_tst_6.shape)

(54879, 4091) (19617, 4091)


In [38]:
X_6[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

#### cnn 모델 학습

In [39]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [40]:
def get_model(number):
    inputs = Input(batch_shape=(None, number, 1))
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(inputs)
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

In [41]:
y = trn.author.values
y.shape

(54879,)

In [ ]:
p_val_ver1 = np.zeros((X_1.shape[0], n_class))
p_tst_ver1 = np.zeros((X_tst_1.shape[0], n_class))
p_val_ver2 = np.zeros((X_2.shape[0], n_class))
p_tst_ver2 = np.zeros((X_tst_2.shape[0], n_class))
p_val_ver3 = np.zeros((X_3.shape[0], n_class))
p_tst_ver3 = np.zeros((X_tst_3.shape[0], n_class))
p_val_ver4 = np.zeros((X_4.shape[0], n_class))
p_tst_ver4 = np.zeros((X_tst_4.shape[0], n_class))
p_val_ver5 = np.zeros((X_5.shape[0], n_class))
p_tst_ver5 = np.zeros((X_tst_5.shape[0], n_class))
p_val_ver6 = np.zeros((X_6.shape[0], n_class))
p_tst_ver6 = np.zeros((X_tst_6.shape[0], n_class))

for X, test in [(X_1, X_tst_1), (X_2, X_tst_2), (X_3, X_tst_3),
               (X_4, X_tst_4), (X_5, X_tst_5), (X_6, X_tst_6)]: 
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        X_train, X_val = X[i_trn], X[i_val]
        y_train, y_val = y[i_trn], y[i_val]
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf = get_model(X.shape[1])
        clf.fit(X[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(X[i_val], to_categorical(y[i_val])),
            epochs=100,
            batch_size=512,
            callbacks=[es])
       
        # Predict
        if X.shape[1]==2685:
            p_val_ver1[i_val, :] = clf.predict(X[i_val])
            p_tst_ver1 += clf.predict(test) / n_class
        elif X.shape[1]==2655:
            p_val_ver2[i_val, :] = clf.predict(X[i_val])
            p_tst_ver2 += clf.predict(test) / n_class
        elif X.shape[1]==1907:
            p_val_ver3[i_val, :] = clf.predict(X[i_val])
            p_tst_ver3 += clf.predict(test) / n_class
        elif X.shape[1]==4720:
            p_val_ver4[i_val, :] = clf.predict(X[i_val])
            p_tst_ver4 += clf.predict(test) / n_class
        elif X.shape[1]==4777:
            p_val_ver5[i_val, :] = clf.predict(X[i_val])
            p_tst_ver5 += clf.predict(test) / n_class
        else:
            p_val_ver6[i_val, :] = clf.predict(X[i_val])
            p_tst_ver6 += clf.predict(test) / n_class
            
        del clf
        clear_session()
        gc.collect()
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver1 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver1, axis=1)) * 100:8.4f}%')
print(f'lr ver1 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver1):8.4f}')
print(f'lr ver2 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver2, axis=1)) * 100:8.4f}%')
print(f'lr ver2 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver2):8.4f}')
print(f'lr ver3 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver3, axis=1)) * 100:8.4f}%')
print(f'lr ver3 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver3):8.4f}')
print(f'lr ver4 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver4, axis=1)) * 100:8.4f}%')
print(f'lr ver4 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver4):8.4f}')
print(f'lr ver5 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver5, axis=1)) * 100:8.4f}%')
print(f'lr ver5 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver5):8.4f}')
print(f'lr ver6 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver6, axis=1)) * 100:8.4f}%')
print(f'lr ver6 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver6):8.4f}')


Training model for CV #1
Epoch 1/100
86/86 [==============================] - 4s 49ms/step - loss: 1.4676 - val_loss: 1.3848
Epoch 2/100
86/86 [==============================] - 4s 45ms/step - loss: 1.3462 - val_loss: 1.3087
Epoch 3/100
86/86 [==============================] - 4s 45ms/step - loss: 1.2915 - val_loss: 1.2892
Epoch 4/100
86/86 [==============================] - 4s 46ms/step - loss: 1.2622 - val_loss: 1.2701
Epoch 5/100
86/86 [==============================] - 4s 45ms/step - loss: 1.2359 - val_loss: 1.2349
Epoch 6/100
86/86 [==============================] - 4s 46ms/step - loss: 1.2239 - val_loss: 1.2327
Epoch 7/100
86/86 [==============================] - 4s 45ms/step - loss: 1.2151 - val_loss: 1.2203
Epoch 8/100
86/86 [==============================] - 4s 45ms/step - loss: 1.1965 - val_loss: 1.2261
Epoch 9/100
86/86 [==============================] - 4s 46ms/step - loss: 1.1899 - val_loss: 1.2120
Epoch 10/100
86/86 [==============================] - 4s 46ms/step - loss: 

Epoch 7/100
86/86 [==============================] - 4s 46ms/step - loss: 1.2629 - val_loss: 1.2692
Epoch 8/100
86/86 [==============================] - 4s 46ms/step - loss: 1.2534 - val_loss: 1.2646
Epoch 9/100
86/86 [==============================] - 4s 47ms/step - loss: 1.2394 - val_loss: 1.2600
Epoch 10/100
86/86 [==============================] - 4s 45ms/step - loss: 1.2332 - val_loss: 1.2432
Epoch 11/100
86/86 [==============================] - 4s 46ms/step - loss: 1.2229 - val_loss: 1.2487
Epoch 12/100
86/86 [==============================] - 4s 46ms/step - loss: 1.2202 - val_loss: 1.2499
Epoch 13/100
86/86 [==============================] - 4s 46ms/step - loss: 1.2168 - val_loss: 1.2470
Epoch 00013: early stopping
Training model for CV #4
Epoch 1/100
86/86 [==============================] - 4s 49ms/step - loss: 1.4960 - val_loss: 1.4313
Epoch 2/100
86/86 [==============================] - 4s 45ms/step - loss: 1.3943 - val_loss: 1.3794
Epoch 3/100
86/86 [========================

Epoch 00022: early stopping
Training model for CV #3
Epoch 1/100
86/86 [==============================] - 3s 37ms/step - loss: 1.5061 - val_loss: 1.4372
Epoch 2/100
86/86 [==============================] - 3s 33ms/step - loss: 1.4279 - val_loss: 1.4126
Epoch 3/100
86/86 [==============================] - 3s 33ms/step - loss: 1.4042 - val_loss: 1.3940
Epoch 4/100
86/86 [==============================] - 3s 33ms/step - loss: 1.3867 - val_loss: 1.3926
Epoch 5/100
86/86 [==============================] - 3s 33ms/step - loss: 1.3716 - val_loss: 1.3772
Epoch 6/100
86/86 [==============================] - 3s 34ms/step - loss: 1.3658 - val_loss: 1.3621
Epoch 7/100
86/86 [==============================] - 3s 34ms/step - loss: 1.3564 - val_loss: 1.3579
Epoch 8/100
86/86 [==============================] - 3s 34ms/step - loss: 1.3488 - val_loss: 1.3530
Epoch 9/100
86/86 [==============================] - 3s 34ms/step - loss: 1.3435 - val_loss: 1.3507
Epoch 10/100
86/86 [===========================

#### 제출 파일 생성 및 기타 파일 생성

In [ ]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver1
sub[sub.columns] = p_tst_ver1
sub.to_csv(sub_ver1_file)

# Ver2
sub[sub.columns] = p_tst_ver2
sub.to_csv(sub_ver2_file)

# Ver3
sub[sub.columns] = p_tst_ver3
sub.to_csv(sub_ver3_file)

# Ver4
sub[sub.columns] = p_tst_ver4
sub.to_csv(sub_ver4_file)

# Ver5
sub[sub.columns] = p_tst_ver5
sub.to_csv(sub_ver5_file)

# Ver6
sub[sub.columns] = p_tst_ver6
sub.to_csv(sub_ver6_file)

In [ ]:
# p_val 파일 생성 -> oof

# Ver1
np.savetxt(p_val_ver1_file, p_val_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_val_ver2_file, p_val_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_val_ver3_file, p_val_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_val_ver4_file, p_val_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_val_ver5_file, p_val_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_val_ver6_file, p_val_ver6, fmt='%.18f', delimiter=',')

In [ ]:
# p_tst 파일 생성 -> test 

# Ver1
np.savetxt(p_tst_ver1_file, p_tst_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_tst_ver2_file, p_tst_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_tst_ver3_file, p_tst_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_tst_ver4_file, p_tst_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_tst_ver5_file, p_tst_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_tst_ver6_file, p_tst_ver6, fmt='%.18f', delimiter=',')

In [ ]:
## DTM 피쳐 생성

- nltk의 word_tokenize 사용

vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_1 = vec.fit_transform(trn['text']).toarray()
X_tst_1 = vec.transform(tst['text']).toarray()
print(X_1.shape, X_tst_1.shape)

X_1[0, :50]

- nltk의 WordPunctTokenizer 사용

vec = CountVectorizer(tokenizer=wordPunctTokenizer.tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_2 = vec.fit_transform(trn['text']).toarray()
X_tst_2 = vec.transform(tst['text']).toarray()
print(X_2.shape, X_tst_2.shape)

X_2[0, :50]

- keras의 text_to_word_sequence 사용

vec = CountVectorizer(tokenizer=text_to_word_sequence, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_3 = vec.fit_transform(trn['text']).toarray()
X_tst_3 = vec.transform(tst['text']).toarray()
print(X_3.shape, X_tst_3.shape)

X_3[0, :50]

- nltk의 word_tokenize 사용 , stopword 없음

vec = CountVectorizer(tokenizer=word_tokenize, ngram_range=(1, 2), min_df=100)
X_4 = vec.fit_transform(trn['text']).toarray()
X_tst_4 = vec.transform(tst['text']).toarray()
print(X_4.shape, X_tst_4.shape)

X_4[0, :50]

- nltk의 WordPunctTokenizer 사용 , stopword 없음

vec = CountVectorizer(tokenizer=wordPunctTokenizer.tokenize, ngram_range=(1, 2), min_df=100)
X_5 = vec.fit_transform(trn['text']).toarray()
X_tst_5 = vec.transform(tst['text']).toarray()
print(X_5.shape, X_tst_5.shape)

X_5[0, :50]

- keras의 text_to_word_sequence 사용 , stopword 없음

vec = CountVectorizer(tokenizer=text_to_word_sequence, ngram_range=(1, 2), min_df=100)
X_6 = vec.fit_transform(trn['text']).toarray()
X_tst_6 = vec.transform(tst['text']).toarray()
print(X_6.shape, X_tst_6.shape)

X_6[0, :50]

## cnn 모델 학습

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

def get_model(number):
    inputs = Input(batch_shape=(None, number, 1))
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(inputs)
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

y = trn.author.values
y.shape

p_val_ver1 = np.zeros((X_1.shape[0], n_class))
p_tst_ver1 = np.zeros((X_tst_1.shape[0], n_class))
p_val_ver2 = np.zeros((X_2.shape[0], n_class))
p_tst_ver2 = np.zeros((X_tst_2.shape[0], n_class))
p_val_ver3 = np.zeros((X_3.shape[0], n_class))
p_tst_ver3 = np.zeros((X_tst_3.shape[0], n_class))
p_val_ver4 = np.zeros((X_4.shape[0], n_class))
p_tst_ver4 = np.zeros((X_tst_4.shape[0], n_class))
p_val_ver5 = np.zeros((X_5.shape[0], n_class))
p_tst_ver5 = np.zeros((X_tst_5.shape[0], n_class))
p_val_ver6 = np.zeros((X_6.shape[0], n_class))
p_tst_ver6 = np.zeros((X_tst_6.shape[0], n_class))

for X, test in [(X_1, X_tst_1), (X_2, X_tst_2), (X_3, X_tst_3),
               (X_4, X_tst_4), (X_5, X_tst_5), (X_6, X_tst_6)]: 
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        X_train, X_val = X[i_trn], X[i_val]
        y_train, y_val = y[i_trn], y[i_val]
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf = get_model(X.shape[1])
        clf.fit(X[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(X[i_val], to_categorical(y[i_val])),
            epochs=100,
            batch_size=512,
            callbacks=[es])
       
        # Predict
        if X.shape[1]==2685:
            p_val_ver1[i_val, :] = clf.predict(X[i_val])
            p_tst_ver1 += clf.predict(test) / n_class
        elif X.shape[1]==2655:
            p_val_ver2[i_val, :] = clf.predict(X[i_val])
            p_tst_ver2 += clf.predict(test) / n_class
        elif X.shape[1]==1907:
            p_val_ver3[i_val, :] = clf.predict(X[i_val])
            p_tst_ver3 += clf.predict(test) / n_class
        elif X.shape[1]==4720:
            p_val_ver4[i_val, :] = clf.predict(X[i_val])
            p_tst_ver4 += clf.predict(test) / n_class
        elif X.shape[1]==4777:
            p_val_ver5[i_val, :] = clf.predict(X[i_val])
            p_tst_ver5 += clf.predict(test) / n_class
        else:
            p_val_ver6[i_val, :] = clf.predict(X[i_val])
            p_tst_ver6 += clf.predict(test) / n_class
            
        del clf
        clear_session()
        gc.collect()
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver1 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver1, axis=1)) * 100:8.4f}%')
print(f'lr ver1 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver1):8.4f}')
print(f'lr ver2 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver2, axis=1)) * 100:8.4f}%')
print(f'lr ver2 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver2):8.4f}')
print(f'lr ver3 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver3, axis=1)) * 100:8.4f}%')
print(f'lr ver3 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver3):8.4f}')
print(f'lr ver4 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver4, axis=1)) * 100:8.4f}%')
print(f'lr ver4 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver4):8.4f}')
print(f'lr ver5 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver5, axis=1)) * 100:8.4f}%')
print(f'lr ver5 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver5):8.4f}')
print(f'lr ver6 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver6, axis=1)) * 100:8.4f}%')
print(f'lr ver6 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver6):8.4f}')


## 제출 파일 생성 및 기타 파일 생성

# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver1
sub[sub.columns] = p_tst_ver1
sub.to_csv(sub_ver1_file)

# Ver2
sub[sub.columns] = p_tst_ver2
sub.to_csv(sub_ver2_file)

# Ver3
sub[sub.columns] = p_tst_ver3
sub.to_csv(sub_ver3_file)

# Ver4
sub[sub.columns] = p_tst_ver4
sub.to_csv(sub_ver4_file)

# Ver5
sub[sub.columns] = p_tst_ver5
sub.to_csv(sub_ver5_file)

# Ver6
sub[sub.columns] = p_tst_ver6
sub.to_csv(sub_ver6_file)

# p_val 파일 생성 -> oof

# Ver1
np.savetxt(p_val_ver1_file, p_val_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_val_ver2_file, p_val_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_val_ver3_file, p_val_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_val_ver4_file, p_val_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_val_ver5_file, p_val_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_val_ver6_file, p_val_ver6, fmt='%.18f', delimiter=',')

# p_tst 파일 생성 -> test 

# Ver1
np.savetxt(p_tst_ver1_file, p_tst_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_tst_ver2_file, p_tst_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_tst_ver3_file, p_tst_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_tst_ver4_file, p_tst_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_tst_ver5_file, p_tst_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_tst_ver6_file, p_tst_ver6, fmt='%.18f', delimiter=',')

### 1-2. Feature

In [ ]:
feature_name = 'feature'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_Ver3_file = feature_dir / f'{feature_name}_Ver3.csv'
feature_Ver4_file = feature_dir / f'{feature_name}_Ver4.csv'
feature_Ver5_file = feature_dir / f'{feature_name}_Ver5.csv'

feature_target_file = feature_dir / f'{feature_name}_target.csv'

#### 변수 생성

- stopwords 제거 X -> Ver1
- stopwords 제거 O -> Ver2
- 중복 Top 100 모두 제거 -> Ver3
- 중복 Top 100중 min*2 <= max 라면 제거 -> Ver4
- tl-idf PCA -> Ver5

In [7]:
# 전테 탑 N에 있는 단어들 가져오는 함수
def get_top_n_vocab(data, top=None):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data)
    
    if top is None:
        vocab_size = len(tokenizer.word_index)
    else:
        vocab_size = min(top,len(tokenizer.word_index))
    
    vocab = {}
    for word, index in tokenizer.word_index.items(): 
        if index == vocab_size+1:
            break
        vocab[word] = (index, tokenizer.word_counts[word])
        
    return vocab

In [8]:
# 전체 탑 100에 있는 단어들 중, 모든 작가에서 발견된 단어들 체크
# 그리고 그 단어들 개수 체크
# 그리고 작가별 단어 빈도수도 체크
def check_vocab_in_author(vocab, vocab_authors):
    cnt = 0 # 모든 작가에서 발견된 단어인지 카운트
    cnt_frequencys = [] # 작가별 빈도수 카운트
    words = {} # 모든 작가에서 발견된 단어들 + 작가별 빈도수
    
    for key in vocab.keys():
        for vocab_author in vocab_authors:
            if key in vocab_author:
                cnt += 1
                cnt_frequencys.append(vocab_author.get(key)[1])
        
        if cnt==5:
            words[key] = tuple(cnt_frequencys) 
            
        cnt = 0
        cnt_frequencys.clear()
    
    return words

In [9]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r"[^A-Za-z0-9' ]", '', text)

# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

In [10]:
df_train = pd.read_csv(trn_file, index_col=0)
df_test = pd.read_csv(tst_file, index_col=0)

In [11]:
#전처리 적용
df_train['text'] = df_train['text'].str.lower()
df_test['text'] = df_test['text'].str.lower()

df_train['text'] = df_train['text'].apply(alpha_num)
df_test['text'] = df_test['text'].apply(alpha_num)

#### Ver 1 생성

In [12]:
df_train_ver1 = df_train.copy()
df_test_ver1 = df_test.copy()

In [13]:
dataset = pd.concat([df_train_ver1,df_test_ver1], axis=0)
dataset.fillna(-1, inplace=True)

In [14]:
feature = pd.DataFrame(dataset)
feature.to_csv(feature_Ver1_file)

#### Ver 2 생성

In [15]:
df_train_ver2 = df_train.copy()
df_test_ver2 = df_test.copy()

In [16]:
# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

df_train_ver2['text'] = df_train_ver2['text'].apply(remove_stopwords)
df_test_ver2['text'] = df_test_ver2['text'].apply(remove_stopwords)

In [17]:
# stopword를 제거 했을 때, 문장 하나가 공백이 되는 현상이 발생함.
# 스태킹을 하기 위해서, 행을 버리는 것은 부담감이 있음.
# 따라서, !!!!를 채운 다음에, 학습 코드에서 제거할 예정
df_train_ver2=df_train_ver2.replace(r'','!!!!')

In [18]:
dataset = pd.concat([df_train_ver2,df_test_ver2], axis=0)
dataset.fillna(-1, inplace=True)

In [19]:
feature = pd.DataFrame(dataset)
feature.to_csv(feature_Ver2_file)

#### Ver3 생성

In [20]:
df_train_ver3 = df_train.copy()
df_test_ver3 = df_test.copy()

In [21]:
# train test 분리
X_train = df_train_ver3['text'].values
X_test = df_test_ver3['text'].values
y_train = df_train_ver3[target_col].values
print(X_train.shape, X_test.shape, y_train.shape)

(54879,) (19617,) (54879,)


In [22]:
all_vocab = get_top_n_vocab(X_train)

author_data = []
for i in range(len(df_train_ver3[target_col].unique())):
    temp = df_train_ver3[df_train_ver3[target_col]==i]['text']
    author_data.append(temp)
    
author_vocab = []
for i in range(len(df_train_ver3[target_col].unique())):
    temp = get_top_n_vocab(author_data[i],100)
    author_vocab.append(temp)
    
check_author_vocab = check_vocab_in_author(all_vocab, author_vocab)

# 불용어
stopwords =[word for word in check_author_vocab.keys()]

df_train_ver3['text'] = df_train_ver3['text'].apply(remove_stopwords)
df_test_ver3['text'] = df_test_ver3['text'].apply(remove_stopwords)

In [35]:
for i,j in enumerate(df_train_ver3['text'].values):
    if type(j) == float:
        print(i,j)

In [ ]:
for i,j in enumerate()

In [23]:
dataset = pd.concat([df_train_ver3,df_test_ver3], axis=0)
dataset.fillna(-1, inplace=True)

In [24]:
feature = pd.DataFrame(dataset)
feature.to_csv(feature_Ver3_file)

#### Ver4 데이터 로드

In [25]:
df_train_ver4 = df_train.copy()
df_test_ver4 = df_test.copy()

In [26]:
# train test 분리
X_train = df_train_ver4['text'].values
X_test = df_test_ver4['text'].values
y_train = df_train_ver4[target_col].values
print(X_train.shape, X_test.shape, y_train.shape)

(54879,) (19617,) (54879,)


In [27]:
all_vocab = get_top_n_vocab(X_train)

author_data = []
for i in range(len(df_train_ver4[target_col].unique())):
    temp = df_train_ver4[df_train_ver4[target_col]==i]['text']
    author_data.append(temp)
    
author_vocab = []
for i in range(len(df_train_ver4[target_col].unique())):
    temp = get_top_n_vocab(author_data[i],100)
    author_vocab.append(temp)
    
check_author_vocab = check_vocab_in_author(all_vocab, author_vocab)

# 불용어
stopwords =[]
for word, feq in check_author_vocab.items():
    word_min, word_max = min(feq), max(feq)
    check = word_min*2 >= word_max
    if check:
        stopwords.append(word)
        
df_train_ver4['text'] = df_train_ver4['text'].apply(remove_stopwords)
df_test_ver4['text'] = df_test_ver4['text'].apply(remove_stopwords)

In [28]:
dataset = pd.concat([df_train_ver4,df_test_ver4], axis=0)
dataset.fillna(-1, inplace=True)

In [29]:
feature = pd.DataFrame(dataset)
feature.to_csv(feature_Ver4_file)

### 1-3. hashing

In [ ]:
algo_name = 'cnn'
feature_name = 'hashing'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'

p_val_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'

p_val_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
p_tst_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'

p_val_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
p_tst_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'

p_val_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'
p_tst_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'

p_val_ver5_file = val_dir / f'{model_name}_oof_pred_ver5.csv'
p_tst_ver5_file = tst_dir / f'{model_name}_test_pred_ver5.csv'

p_val_ver6_file = val_dir / f'{model_name}_oof_pred_ver6.csv'
p_tst_ver6_file = tst_dir / f'{model_name}_test_pred_ver6.csv'

sub_ver1_file = sub_dir / f'{model_name}_ver1.csv'
sub_ver2_file = sub_dir / f'{model_name}_ver2.csv'
sub_ver3_file = sub_dir / f'{model_name}_ver3.csv'
sub_ver4_file = sub_dir / f'{model_name}_ver4.csv'
sub_ver5_file = sub_dir / f'{model_name}_ver5.csv'
sub_ver6_file = sub_dir / f'{model_name}_ver6.csv'

In [ ]:
## Hashing 피쳐 생성

- nltk의 word_tokenize 사용

vec = HashingVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), n_features=2**10)
X_1 = vec.fit_transform(trn['text']).toarray()
X_tst_1 = vec.transform(tst['text']).toarray()
print(X_1.shape, X_tst_1.shape)

X_1[0, :50]

- nltk의 WordPunctTokenizer 사용

vec = HashingVectorizer(tokenizer=wordPunctTokenizer.tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), n_features=2**10)
X_2 = vec.fit_transform(trn['text']).toarray()
X_tst_2 = vec.transform(tst['text']).toarray()
print(X_2.shape, X_tst_2.shape)

X_2[0, :50]

- keras의 text_to_word_sequence 사용

vec = HashingVectorizer(tokenizer=text_to_word_sequence, stop_words=stopwords.words('english'), ngram_range=(1, 3), n_features=2**10)
X_3 = vec.fit_transform(trn['text']).toarray()
X_tst_3 = vec.transform(tst['text']).toarray()
print(X_3.shape, X_tst_3.shape)

X_3[0, :50]

- nltk의 word_tokenize 사용, stopword 제거

vec = HashingVectorizer(tokenizer=word_tokenize, ngram_range=(1, 3), n_features=2**10)
X_4 = vec.fit_transform(trn['text']).toarray()
X_tst_4 = vec.transform(tst['text']).toarray()
print(X_4.shape, X_tst_4.shape)

X_4[0, :50]

- nltk의 WordPunctTokenizer 사용, stopword 제거

vec = HashingVectorizer(tokenizer=wordPunctTokenizer.tokenize, ngram_range=(1, 3), n_features=2**10)
X_5 = vec.fit_transform(trn['text']).toarray()
X_tst_5 = vec.transform(tst['text']).toarray()
print(X_5.shape, X_tst_5.shape)

X_5[0, :50]

- keras의 text_to_word_sequence 사용, stopword 제거

vec = HashingVectorizer(tokenizer=text_to_word_sequence, ngram_range=(1, 3), n_features=2**10)
X_6 = vec.fit_transform(trn['text']).toarray()
X_tst_6 = vec.transform(tst['text']).toarray()
print(X_6.shape, X_tst_6.shape)

X_6[0, :50]

## cnn 모델 학습

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

def get_model(number):
    inputs = Input(batch_shape=(None, number, 1))
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(inputs)
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

y = trn.author.values
y.shape

p_val_ver1 = np.zeros((X_1.shape[0], n_class))
p_tst_ver1 = np.zeros((X_tst_1.shape[0], n_class))
p_val_ver2 = np.zeros((X_2.shape[0], n_class))
p_tst_ver2 = np.zeros((X_tst_2.shape[0], n_class))
p_val_ver3 = np.zeros((X_3.shape[0], n_class))
p_tst_ver3 = np.zeros((X_tst_3.shape[0], n_class))
p_val_ver4 = np.zeros((X_4.shape[0], n_class))
p_tst_ver4 = np.zeros((X_tst_4.shape[0], n_class))
p_val_ver5 = np.zeros((X_5.shape[0], n_class))
p_tst_ver5 = np.zeros((X_tst_5.shape[0], n_class))
p_val_ver6 = np.zeros((X_6.shape[0], n_class))
p_tst_ver6 = np.zeros((X_tst_6.shape[0], n_class))

for number, (X, test) in enumerate([(X_1, X_tst_1), (X_2, X_tst_2), (X_3, X_tst_3),
               (X_4, X_tst_4), (X_5, X_tst_5), (X_6, X_tst_6)],1):
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf = get_model(X.shape[1])
        clf.fit(X[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(X[i_val], to_categorical(y[i_val])),
            epochs=100,
            batch_size=512,
            callbacks=[es])
       
        # Predict
        if number==1:
            p_val_ver1[i_val, :] = clf.predict(X[i_val])
            p_tst_ver1 += clf.predict(test) / n_class
        elif number==2:
            p_val_ver2[i_val, :] = clf.predict(X[i_val])
            p_tst_ver2 += clf.predict(test) / n_class
        elif number==3:
            p_val_ver3[i_val, :] = clf.predict(X[i_val])
            p_tst_ver3 += clf.predict(test) / n_class
        elif number==4:
            p_val_ver4[i_val, :] = clf.predict(X[i_val])
            p_tst_ver4 += clf.predict(test) / n_class
        elif number==5:
            p_val_ver5[i_val, :] = clf.predict(X[i_val])
            p_tst_ver5 += clf.predict(test) / n_class
        else:
            p_val_ver6[i_val, :] = clf.predict(X[i_val])
            p_tst_ver6 += clf.predict(test) / n_class
            
        del clf
        clear_session()
        gc.collect()
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver1 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver1, axis=1)) * 100:8.4f}%')
print(f'lr ver1 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver1):8.4f}')
print(f'lr ver2 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver2, axis=1)) * 100:8.4f}%')
print(f'lr ver2 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver2):8.4f}')
print(f'lr ver3 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver3, axis=1)) * 100:8.4f}%')
print(f'lr ver3 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver3):8.4f}')
print(f'lr ver4 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver4, axis=1)) * 100:8.4f}%')
print(f'lr ver4 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver4):8.4f}')
print(f'lr ver5 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver5, axis=1)) * 100:8.4f}%')
print(f'lr ver5 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver5):8.4f}')
print(f'lr ver6 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver6, axis=1)) * 100:8.4f}%')
print(f'lr ver6 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver6):8.4f}')

## 제출 파일 생성 및 기타 파일 생성

# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver1
sub[sub.columns] = p_tst_ver1
sub.to_csv(sub_ver1_file)

# Ver2
sub[sub.columns] = p_tst_ver2
sub.to_csv(sub_ver2_file)

# Ver3
sub[sub.columns] = p_tst_ver3
sub.to_csv(sub_ver3_file)

# Ver4
sub[sub.columns] = p_tst_ver4
sub.to_csv(sub_ver4_file)

# Ver5
sub[sub.columns] = p_tst_ver5
sub.to_csv(sub_ver5_file)

# Ver6
sub[sub.columns] = p_tst_ver6
sub.to_csv(sub_ver6_file)

# p_val 파일 생성 -> oof

# Ver1
np.savetxt(p_val_ver1_file, p_val_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_val_ver2_file, p_val_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_val_ver3_file, p_val_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_val_ver4_file, p_val_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_val_ver5_file, p_val_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_val_ver6_file, p_val_ver6, fmt='%.18f', delimiter=',')

# p_tst 파일 생성 -> test 

# Ver1
np.savetxt(p_tst_ver1_file, p_tst_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_tst_ver2_file, p_tst_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_tst_ver3_file, p_tst_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_tst_ver4_file, p_tst_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_tst_ver5_file, p_tst_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_tst_ver6_file, p_tst_ver6, fmt='%.18f', delimiter=',')

### 1-4. tf-idf

In [ ]:
algo_name = 'cnn'
feature_name = 'tfidf'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'

p_val_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'

p_val_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
p_tst_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'

p_val_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
p_tst_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'

p_val_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'
p_tst_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'

p_val_ver5_file = val_dir / f'{model_name}_oof_pred_ver5.csv'
p_tst_ver5_file = tst_dir / f'{model_name}_test_pred_ver5.csv'

p_val_ver6_file = val_dir / f'{model_name}_oof_pred_ver6.csv'
p_tst_ver6_file = tst_dir / f'{model_name}_test_pred_ver6.csv'

sub_ver1_file = sub_dir / f'{model_name}_ver1.csv'
sub_ver2_file = sub_dir / f'{model_name}_ver2.csv'
sub_ver3_file = sub_dir / f'{model_name}_ver3.csv'
sub_ver4_file = sub_dir / f'{model_name}_ver4.csv'
sub_ver5_file = sub_dir / f'{model_name}_ver5.csv'
sub_ver6_file = sub_dir / f'{model_name}_ver6.csv'

#### TF-IDF 피쳐 생성

- nltk의 word_tokenize 사용

In [27]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_1 = vec.fit_transform(trn['text']).toarray()
X_tst_1 = vec.transform(tst['text']).toarray()
print(X_1.shape, X_tst_1.shape)

(54879, 5897) (19617, 5897)


In [28]:
X_1[0, :50]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

- nltk의 WordPunctTokenizer 사용

In [29]:
vec = TfidfVectorizer(tokenizer=wordPunctTokenizer.tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_2 = vec.fit_transform(trn['text']).toarray()
X_tst_2 = vec.transform(tst['text']).toarray()
print(X_2.shape, X_tst_2.shape)

(54879, 5772) (19617, 5772)


In [30]:
X_2[0, :50]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

- keras의 text_to_word_sequence 사용

In [31]:
vec = TfidfVectorizer(tokenizer=text_to_word_sequence, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_3 = vec.fit_transform(trn['text']).toarray()
X_tst_3 = vec.transform(tst['text']).toarray()
print(X_3.shape, X_tst_3.shape)

(54879, 3745) (19617, 3745)


In [32]:
X_3[0, :50]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

- nltk의 word_tokenize 사용, stopword 제거

In [33]:
vec = TfidfVectorizer(tokenizer=word_tokenize, ngram_range=(1, 3), min_df=50)
X_4 = vec.fit_transform(trn['text']).toarray()
X_tst_4 = vec.transform(tst['text']).toarray()
print(X_4.shape, X_tst_4.shape)

(54879, 12254) (19617, 12254)


In [34]:
X_4[0, :50]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

- nltk의 WordPunctTokenizer 사용, stopword 제거

In [ ]:
vec = TfidfVectorizer(tokenizer=wordPunctTokenizer.tokenize, ngram_range=(1, 3), min_df=50)
X_5 = vec.fit_transform(trn['text']).toarray()
X_tst_5 = vec.transform(tst['text']).toarray()
print(X_5.shape, X_tst_5.shape)

In [ ]:
X_5[0, :50]

- keras의 text_to_word_sequence 사용, stopword 제거

In [ ]:
vec = TfidfVectorizer(tokenizer=text_to_word_sequence, ngram_range=(1, 3), min_df=50)
X_6 = vec.fit_transform(trn['text']).toarray()
X_tst_6 = vec.transform(tst['text']).toarray()
print(X_6.shape, X_tst_6.shape)

In [ ]:
X_6[0, :50]

#### cnn 모델 학습

In [ ]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [ ]:
def get_model(number):
    inputs = Input(batch_shape=(None, number, 1))
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(inputs)
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

In [ ]:
y = trn.author.values
y.shape

In [42]:
p_val_ver1 = np.zeros((X_1.shape[0], n_class))
p_tst_ver1 = np.zeros((X_tst_1.shape[0], n_class))
p_val_ver2 = np.zeros((X_2.shape[0], n_class))
p_tst_ver2 = np.zeros((X_tst_2.shape[0], n_class))
p_val_ver3 = np.zeros((X_3.shape[0], n_class))
p_tst_ver3 = np.zeros((X_tst_3.shape[0], n_class))
p_val_ver4 = np.zeros((X_4.shape[0], n_class))
p_tst_ver4 = np.zeros((X_tst_4.shape[0], n_class))
p_val_ver5 = np.zeros((X_5.shape[0], n_class))
p_tst_ver5 = np.zeros((X_tst_5.shape[0], n_class))
p_val_ver6 = np.zeros((X_6.shape[0], n_class))
p_tst_ver6 = np.zeros((X_tst_6.shape[0], n_class))

for X, test in [(X_1, X_tst_1), (X_2, X_tst_2), (X_3, X_tst_3),
               (X_4, X_tst_4), (X_5, X_tst_5), (X_6, X_tst_6)]:  
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        X = X.reshape(X.shape[0], X.shape[1], 1)
        test = test.reshape(test.shape[0], test.shape[1],1)
        
        clf = get_model(X.shape[1])
        clf.fit(X[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(X[i_val], to_categorical(y[i_val])),
            epochs=100,
            batch_size=256,
            callbacks=[es])
       
        # Predict
        if X.shape[1]==5897:
            p_val_ver1[i_val, :] = clf.predict(X[i_val])
            p_tst_ver1 += clf.predict(test) / n_class
        elif X.shape[1]==5772:
            p_val_ver2[i_val, :] = clf.predict(X[i_val])
            p_tst_ver2 += clf.predict(test) / n_class
        elif X.shape[1]==3745:
            p_val_ver3[i_val, :] = clf.predict(X[i_val])
            p_tst_ver3 += clf.predict(test) / n_class
        elif X.shape[1]==12254:
            p_val_ver4[i_val, :] = clf.predict(X[i_val])
            p_tst_ver4 += clf.predict(test) / n_class
        elif X.shape[1]==12267:
            p_val_ver5[i_val, :] = clf.predict(X[i_val])
            p_tst_ver5 += clf.predict(test) / n_class
        else:
            p_val_ver6[i_val, :] = clf.predict(X[i_val])
            p_tst_ver6 += clf.predict(test) / n_class
            
        del clf
        clear_session()
        gc.collect()
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver1 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver1, axis=1)) * 100:8.4f}%')
print(f'lr ver1 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver1):8.4f}')
print(f'lr ver2 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver2, axis=1)) * 100:8.4f}%')
print(f'lr ver2 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver2):8.4f}')
print(f'lr ver3 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver3, axis=1)) * 100:8.4f}%')
print(f'lr ver3 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver3):8.4f}')
print(f'lr ver4 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver4, axis=1)) * 100:8.4f}%')
print(f'lr ver4 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver4):8.4f}')
print(f'lr ver5 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver5, axis=1)) * 100:8.4f}%')
print(f'lr ver5 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver5):8.4f}')
print(f'lr ver6 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver6, axis=1)) * 100:8.4f}%')
print(f'lr ver6 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver6):8.4f}')

Training model for CV #1
Epoch 1/100
172/172 [==============================] - 9s 52ms/step - loss: 1.5582 - val_loss: 1.5428
Epoch 2/100
172/172 [==============================] - 8s 49ms/step - loss: 1.5275 - val_loss: 1.5036
Epoch 3/100
172/172 [==============================] - 8s 49ms/step - loss: 1.4911 - val_loss: 1.4750
Epoch 4/100
172/172 [==============================] - 8s 49ms/step - loss: 1.4591 - val_loss: 1.4610
Epoch 5/100
172/172 [==============================] - 8s 49ms/step - loss: 1.4320 - val_loss: 1.4364
Epoch 6/100
172/172 [==============================] - 8s 49ms/step - loss: 1.4079 - val_loss: 1.4354
Epoch 7/100
172/172 [==============================] - 8s 49ms/step - loss: 1.3813 - val_loss: 1.3942
Epoch 8/100
172/172 [==============================] - 8s 49ms/step - loss: 1.3502 - val_loss: 1.3609
Epoch 9/100
172/172 [==============================] - 8s 49ms/step - loss: 1.3321 - val_loss: 1.3436
Epoch 10/100
172/172 [==============================] - 8

172/172 [==============================] - 9s 51ms/step - loss: 1.5622 - val_loss: 1.5459
Epoch 2/100
172/172 [==============================] - 8s 48ms/step - loss: 1.5275 - val_loss: 1.5104
Epoch 3/100
172/172 [==============================] - 8s 49ms/step - loss: 1.4918 - val_loss: 1.4874
Epoch 4/100
172/172 [==============================] - 8s 49ms/step - loss: 1.4615 - val_loss: 1.4546
Epoch 5/100
172/172 [==============================] - 8s 48ms/step - loss: 1.4402 - val_loss: 1.4381
Epoch 6/100
172/172 [==============================] - 8s 48ms/step - loss: 1.4247 - val_loss: 1.4476
Epoch 7/100
172/172 [==============================] - 8s 48ms/step - loss: 1.4098 - val_loss: 1.4412
Epoch 8/100
172/172 [==============================] - 8s 49ms/step - loss: 1.3963 - val_loss: 1.4186
Epoch 9/100
172/172 [==============================] - 8s 48ms/step - loss: 1.3813 - val_loss: 1.3952
Epoch 10/100
172/172 [==============================] - 8s 48ms/step - loss: 1.3693 - val_loss

Epoch 1/100
172/172 [==============================] - 6s 35ms/step - loss: 1.5550 - val_loss: 1.5242
Epoch 2/100
172/172 [==============================] - 6s 33ms/step - loss: 1.5104 - val_loss: 1.4897
Epoch 3/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4846 - val_loss: 1.4727
Epoch 4/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4674 - val_loss: 1.4610
Epoch 5/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4554 - val_loss: 1.4536
Epoch 6/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4460 - val_loss: 1.4471
Epoch 7/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4357 - val_loss: 1.4433
Epoch 8/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4292 - val_loss: 1.4291
Epoch 9/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4189 - val_loss: 1.4345
Epoch 10/100
172/172 [==============================] - 6s 33ms/step - loss: 1.410

172/172 [==============================] - 6s 35ms/step - loss: 1.5593 - val_loss: 1.5367
Epoch 2/100
172/172 [==============================] - 6s 33ms/step - loss: 1.5127 - val_loss: 1.5029
Epoch 3/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4841 - val_loss: 1.4873
Epoch 4/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4685 - val_loss: 1.4718
Epoch 5/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4555 - val_loss: 1.4670
Epoch 6/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4485 - val_loss: 1.4688
Epoch 7/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4380 - val_loss: 1.4497
Epoch 8/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4304 - val_loss: 1.4465
Epoch 9/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4221 - val_loss: 1.4380
Epoch 10/100
172/172 [==============================] - 6s 33ms/step - loss: 1.4170 - val_loss

Epoch 00023: early stopping
Training model for CV #4
Epoch 1/100
172/172 [==============================] - 17s 100ms/step - loss: 1.5643 - val_loss: 1.5556
Epoch 2/100
172/172 [==============================] - 17s 96ms/step - loss: 1.5474 - val_loss: 1.5404
Epoch 3/100
172/172 [==============================] - 17s 97ms/step - loss: 1.5216 - val_loss: 1.5148
Epoch 4/100
172/172 [==============================] - 17s 97ms/step - loss: 1.5015 - val_loss: 1.5075
Epoch 5/100
172/172 [==============================] - 17s 97ms/step - loss: 1.4795 - val_loss: 1.4816
Epoch 6/100
172/172 [==============================] - 17s 97ms/step - loss: 1.4619 - val_loss: 1.4755
Epoch 7/100
172/172 [==============================] - 17s 98ms/step - loss: 1.4425 - val_loss: 1.4559
Epoch 8/100
172/172 [==============================] - 17s 97ms/step - loss: 1.4232 - val_loss: 1.4449
Epoch 9/100
172/172 [==============================] - 17s 97ms/step - loss: 1.4072 - val_loss: 1.4391
Epoch 10/100
172/17

172/172 [==============================] - 17s 98ms/step - loss: 1.2945 - val_loss: 1.3610
Epoch 21/100
172/172 [==============================] - 17s 97ms/step - loss: 1.2913 - val_loss: 1.3483
Epoch 22/100
172/172 [==============================] - 17s 98ms/step - loss: 1.2858 - val_loss: 1.3496
Epoch 23/100
172/172 [==============================] - 17s 97ms/step - loss: 1.2794 - val_loss: 1.3526
Epoch 24/100
172/172 [==============================] - 17s 97ms/step - loss: 1.2735 - val_loss: 1.3531
Epoch 00024: early stopping
Training model for CV #2
Epoch 1/100
172/172 [==============================] - 17s 100ms/step - loss: 1.5654 - val_loss: 1.5565
Epoch 2/100
172/172 [==============================] - 17s 97ms/step - loss: 1.5475 - val_loss: 1.5408
Epoch 3/100
172/172 [==============================] - 17s 98ms/step - loss: 1.5251 - val_loss: 1.5157
Epoch 4/100
172/172 [==============================] - 17s 97ms/step - loss: 1.4944 - val_loss: 1.4935
Epoch 5/100
172/172 [======

Epoch 23/100
172/172 [==============================] - 14s 79ms/step - loss: 1.3356 - val_loss: 1.4015
Epoch 24/100
172/172 [==============================] - 14s 79ms/step - loss: 1.3333 - val_loss: 1.3961
Epoch 00024: early stopping
Training model for CV #2
Epoch 1/100
172/172 [==============================] - 14s 82ms/step - loss: 1.5614 - val_loss: 1.5488
Epoch 2/100
172/172 [==============================] - 13s 78ms/step - loss: 1.5307 - val_loss: 1.5311
Epoch 3/100
172/172 [==============================] - 14s 79ms/step - loss: 1.5109 - val_loss: 1.5088
Epoch 4/100
172/172 [==============================] - 14s 79ms/step - loss: 1.4972 - val_loss: 1.4983
Epoch 5/100
172/172 [==============================] - 14s 79ms/step - loss: 1.4863 - val_loss: 1.4886
Epoch 6/100
172/172 [==============================] - 14s 79ms/step - loss: 1.4738 - val_loss: 1.4849
Epoch 7/100
172/172 [==============================] - 14s 79ms/step - loss: 1.4677 - val_loss: 1.4752
Epoch 8/100
172/17

172/172 [==============================] - 14s 79ms/step - loss: 1.3669 - val_loss: 1.4204
Epoch 17/100
172/172 [==============================] - 14s 79ms/step - loss: 1.3624 - val_loss: 1.4088
Epoch 18/100
172/172 [==============================] - 14s 79ms/step - loss: 1.3592 - val_loss: 1.4070
Epoch 19/100
172/172 [==============================] - 14s 79ms/step - loss: 1.3546 - val_loss: 1.3970
Epoch 20/100
172/172 [==============================] - 14s 79ms/step - loss: 1.3488 - val_loss: 1.3982
Epoch 21/100
172/172 [==============================] - 14s 79ms/step - loss: 1.3438 - val_loss: 1.3963
Epoch 22/100
172/172 [==============================] - 14s 79ms/step - loss: 1.3369 - val_loss: 1.3907
Epoch 23/100
172/172 [==============================] - 14s 79ms/step - loss: 1.3356 - val_loss: 1.3982
Epoch 24/100
172/172 [==============================] - 14s 79ms/step - loss: 1.3300 - val_loss: 1.3928
Epoch 25/100
172/172 [==============================] - 14s 79ms/step - loss:

#### 제출 파일 생성 및 기타 파일 생성

In [43]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver1
sub[sub.columns] = p_tst_ver1
sub.to_csv(sub_ver1_file)

# Ver2
sub[sub.columns] = p_tst_ver2
sub.to_csv(sub_ver2_file)

# Ver3
sub[sub.columns] = p_tst_ver3
sub.to_csv(sub_ver3_file)

# Ver4
sub[sub.columns] = p_tst_ver4
sub.to_csv(sub_ver4_file)

# Ver5
sub[sub.columns] = p_tst_ver5
sub.to_csv(sub_ver5_file)

# Ver6
sub[sub.columns] = p_tst_ver6
sub.to_csv(sub_ver6_file)

In [44]:
# p_val 파일 생성 -> oof

# Ver1
np.savetxt(p_val_ver1_file, p_val_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_val_ver2_file, p_val_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_val_ver3_file, p_val_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_val_ver4_file, p_val_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_val_ver5_file, p_val_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_val_ver6_file, p_val_ver6, fmt='%.18f', delimiter=',')

In [45]:
# p_tst 파일 생성 -> test 

# Ver1
np.savetxt(p_tst_ver1_file, p_tst_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_tst_ver2_file, p_tst_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_tst_ver3_file, p_tst_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_tst_ver4_file, p_tst_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_tst_ver5_file, p_tst_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_tst_ver6_file, p_tst_ver6, fmt='%.18f', delimiter=',')

### 1-5. tf-idf-pca

In [23]:
algo_name = 'xgboost'
feature_name = 'tfidf-pca'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_file =  tst_dir / f'{model_name}_test_pred_ver1.csv'
sub_file = sub_dir / f'{model_name}_submission_ver1.csv'

In [25]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
#nltk.download('t')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Bag-of-Words 피처 생성

In [26]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_cnt = vec.fit_transform(trn['text'])
print(X_cnt.shape)

(54879, 2685)


In [27]:
X_cnt[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

In [28]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X = vec.fit_transform(trn['text'])
X_tst = vec.transform(tst['text'])
print(X.shape, X_tst.shape)

(54879, 5897) (19617, 5897)


In [29]:
X[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

# PCA

In [32]:
n_batches = 39
inc_pca = IncrementalPCA(n_components=500)#temp.size*0.5)
for batch_x in np.array_split(X.todense(), n_batches):
    print(".", end="") # not shown in the book
    inc_pca.partial_fit(batch_x)

# incrementalPCA로 PCA 값을 생성
X_inc = inc_pca.transform(X.todense())    

inc_pca = IncrementalPCA(n_components=500)#temp.size*0.5)
for batch_x in np.array_split(X_tst.todense(), n_batches):
    print(".", end="") # not shown in the book
    inc_pca.partial_fit(batch_x)
    
# incrementalPCA로 PCA 값을 생성
X_tst_inc = inc_pca.transform(X_tst.todense())

..............................................................................

In [35]:
a = pd.concat([pd.DataFrame(X_inc), pd.DataFrame(X_tst_inc)])
a.to_csv(feature_file)

In [36]:
algo_name = 'cnn'
feature_name = 'tfidf-pca' #학습할 피처- tfidf 피처를 pca한 데이터
feature_target = 'feature_target'

model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv' #학습할 피처 가져오기
feature_target_file = feature_dir / f'{feature_target}.csv' #target 데이터 가져오기

p_val_ver7_file = val_dir / f'{model_name}_oof_pred_ver7.csv'
p_tst_ver7_file = tst_dir / f'{model_name}_test_pred_ver7.csv'
sub_ver7_file = sub_dir / f'{model_name}_ver7.csv'

#### 데이터 셋 불러오기

In [37]:
X = pd.read_csv(feature_file, index_col=0)
y = pd.read_csv(feature_target_file,index_col=0).values

X_7 = X.iloc[:54879].values
X_tst_7 = X.iloc[54879:].values

#### cnn 모델 학습

In [38]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [39]:
def get_model(number):
    inputs = Input(batch_shape=(None, number, 1))
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(inputs)
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

In [40]:
p_val_ver7 = np.zeros((X_7.shape[0], n_class))
p_tst_ver7 = np.zeros((X_tst_7.shape[0], n_class))


for X, test in [(X_7, X_tst_7)]:  
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        X = X.reshape(X.shape[0], X.shape[1], 1)
        test = test.reshape(test.shape[0], test.shape[1],1)
        
        clf = get_model(X.shape[1])
        clf.fit(X[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(X[i_val], to_categorical(y[i_val])),
            epochs=100,
            batch_size=64,
            callbacks=[es])
       
        # Predict
        if X.shape[1]==500:
            p_val_ver7[i_val, :] = clf.predict(X[i_val])
            p_tst_ver7 += clf.predict(test) / n_class
        
            
        del clf
        clear_session()
        gc.collect()
            
    print("Training has finished")
    print("*"*100)

            



Training model for CV #1
Epoch 1/100
686/686 [==============================] - 3s 5ms/step - loss: 1.3255 - val_loss: 1.1599
Epoch 2/100
686/686 [==============================] - 3s 4ms/step - loss: 1.1015 - val_loss: 1.0833
Epoch 3/100
686/686 [==============================] - 3s 4ms/step - loss: 1.0338 - val_loss: 1.0992
Epoch 4/100
686/686 [==============================] - 3s 4ms/step - loss: 0.9902 - val_loss: 1.0488
Epoch 5/100
686/686 [==============================] - 3s 4ms/step - loss: 0.9602 - val_loss: 1.0068
Epoch 6/100
686/686 [==============================] - 3s 4ms/step - loss: 0.9321 - val_loss: 1.0124
Epoch 7/100
686/686 [==============================] - 3s 4ms/step - loss: 0.9092 - val_loss: 1.0017
Epoch 8/100
686/686 [==============================] - 3s 4ms/step - loss: 0.8914 - val_loss: 1.0168
Epoch 9/100
686/686 [==============================] - 3s 4ms/step - loss: 0.8700 - val_loss: 0.9846
Epoch 10/100
686/686 [==============================] - 3s 4ms/ste

686/686 [==============================] - 3s 4ms/step - loss: 0.7253 - val_loss: 1.0671
Epoch 20/100
686/686 [==============================] - 3s 4ms/step - loss: 0.7096 - val_loss: 1.0109
Epoch 21/100
686/686 [==============================] - 3s 4ms/step - loss: 0.6995 - val_loss: 1.0253
Epoch 00021: early stopping
Training model for CV #5
Epoch 1/100
686/686 [==============================] - 3s 4ms/step - loss: 1.3330 - val_loss: 1.1666
Epoch 2/100
686/686 [==============================] - 3s 4ms/step - loss: 1.1078 - val_loss: 1.0945
Epoch 3/100
686/686 [==============================] - 3s 4ms/step - loss: 1.0389 - val_loss: 1.0573
Epoch 4/100
686/686 [==============================] - 3s 4ms/step - loss: 0.9987 - val_loss: 1.0261
Epoch 5/100
686/686 [==============================] - 3s 4ms/step - loss: 0.9662 - val_loss: 1.0059
Epoch 6/100
686/686 [==============================] - 3s 4ms/step - loss: 0.9404 - val_loss: 1.0080
Epoch 7/100
686/686 [===========================

In [41]:
print(f'lr ver7 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver7, axis=1)) * 100:8.4f}%')
print(f'lr ver7 Log Loss (CV): {log_loss(y, p_val_ver7):8.4f}')

lr ver7 Accuracy (CV):  62.1312%
lr ver7 Log Loss (CV):   0.9748


#### 제출 파일 생성 및 기타 파일 생성

In [42]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver7
sub[sub.columns] = p_tst_ver7
sub.to_csv(sub_ver7_file)

In [43]:
# p_val 파일 생성 -> oof

# Ver7
np.savetxt(p_val_ver7_file, p_val_ver7, fmt='%.18f', delimiter=',')

In [44]:
# p_tst 파일 생성 -> test 

# Ver7
np.savetxt(p_tst_ver7_file, p_tst_ver7, fmt='%.18f', delimiter=',')

## 2. MLP

### 2-1. bow

In [ ]:
algo_name = 'mlp'
feature_name = 'bow'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'

p_val_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'

p_val_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
p_tst_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'

p_val_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
p_tst_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'

p_val_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'
p_tst_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'

p_val_ver5_file = val_dir / f'{model_name}_oof_pred_ver5.csv'
p_tst_ver5_file = tst_dir / f'{model_name}_test_pred_ver5.csv'

p_val_ver6_file = val_dir / f'{model_name}_oof_pred_ver6.csv'
p_tst_ver6_file = tst_dir / f'{model_name}_test_pred_ver6.csv'

sub_ver1_file = sub_dir / f'{model_name}_ver1.csv'
sub_ver2_file = sub_dir / f'{model_name}_ver2.csv'
sub_ver3_file = sub_dir / f'{model_name}_ver3.csv'
sub_ver4_file = sub_dir / f'{model_name}_ver4.csv'
sub_ver5_file = sub_dir / f'{model_name}_ver5.csv'
sub_ver6_file = sub_dir / f'{model_name}_ver6.csv'

#### DTM 피쳐 생성

- nltk의 word_tokenize 사용

In [27]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_1 = vec.fit_transform(trn['text']).toarray()
X_tst_1 = vec.transform(tst['text']).toarray()
print(X_1.shape, X_tst_1.shape)

(54879, 2685) (19617, 2685)


In [28]:
X_1[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

- nltk의 WordPunctTokenizer 사용

In [29]:
vec = CountVectorizer(tokenizer=wordPunctTokenizer.tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_2 = vec.fit_transform(trn['text']).toarray()
X_tst_2 = vec.transform(tst['text']).toarray()
print(X_2.shape, X_tst_2.shape)

(54879, 2655) (19617, 2655)


In [30]:
X_2[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 4, 0, 0, 0])

- keras의 text_to_word_sequence 사용

In [31]:
vec = CountVectorizer(tokenizer=text_to_word_sequence, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_3 = vec.fit_transform(trn['text']).toarray()
X_tst_3 = vec.transform(tst['text']).toarray()
print(X_3.shape, X_tst_3.shape)

(54879, 1907) (19617, 1907)


In [32]:
X_3[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

- nltk의 word_tokenize 사용 , stopword 없음

In [33]:
vec = CountVectorizer(tokenizer=word_tokenize, ngram_range=(1, 2), min_df=100)
X_4 = vec.fit_transform(trn['text']).toarray()
X_tst_4 = vec.transform(tst['text']).toarray()
print(X_4.shape, X_tst_4.shape)

(54879, 4720) (19617, 4720)


In [34]:
X_4[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

- nltk의 WordPunctTokenizer 사용 , stopword 없음

In [35]:
vec = CountVectorizer(tokenizer=wordPunctTokenizer.tokenize, ngram_range=(1, 2), min_df=100)
X_5 = vec.fit_transform(trn['text']).toarray()
X_tst_5 = vec.transform(tst['text']).toarray()
print(X_5.shape, X_tst_5.shape)

(54879, 4777) (19617, 4777)


In [36]:
X_5[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

- keras의 text_to_word_sequence 사용 , stopword 없음

In [37]:
vec = CountVectorizer(tokenizer=text_to_word_sequence, ngram_range=(1, 2), min_df=100)
X_6 = vec.fit_transform(trn['text']).toarray()
X_tst_6 = vec.transform(tst['text']).toarray()
print(X_6.shape, X_tst_6.shape)

(54879, 4091) (19617, 4091)


In [38]:
X_6[0, :50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

#### mlp 모델 학습

In [39]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [40]:
def get_model(number):
    inputs = Input(shape=(number,))
    x = Dense(128, activation='relu')(inputs)
    outputs = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

In [43]:
y = trn.author.values
y.shape

(54879,)

In [44]:
p_val_ver1 = np.zeros((X_1.shape[0], n_class))
p_tst_ver1 = np.zeros((X_tst_1.shape[0], n_class))
p_val_ver2 = np.zeros((X_2.shape[0], n_class))
p_tst_ver2 = np.zeros((X_tst_2.shape[0], n_class))
p_val_ver3 = np.zeros((X_3.shape[0], n_class))
p_tst_ver3 = np.zeros((X_tst_3.shape[0], n_class))
p_val_ver4 = np.zeros((X_4.shape[0], n_class))
p_tst_ver4 = np.zeros((X_tst_4.shape[0], n_class))
p_val_ver5 = np.zeros((X_5.shape[0], n_class))
p_tst_ver5 = np.zeros((X_tst_5.shape[0], n_class))
p_val_ver6 = np.zeros((X_6.shape[0], n_class))
p_tst_ver6 = np.zeros((X_tst_6.shape[0], n_class))

for X, test in [(X_1, X_tst_1), (X_2, X_tst_2), (X_3, X_tst_3),
               (X_4, X_tst_4), (X_5, X_tst_5), (X_6, X_tst_6)]: 
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        X_train, X_val = X[i_trn], X[i_val]
        y_train, y_val = y[i_trn], y[i_val]
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf = get_model(X.shape[1])
        clf.fit(X[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(X[i_val], to_categorical(y[i_val])),
            epochs=100,
            batch_size=512,
            callbacks=[es])
       
        # Predict
        if X.shape[1]==2685:
            p_val_ver1[i_val, :] = clf.predict(X[i_val])
            p_tst_ver1 += clf.predict(test) / n_class
        elif X.shape[1]==2655:
            p_val_ver2[i_val, :] = clf.predict(X[i_val])
            p_tst_ver2 += clf.predict(test) / n_class
        elif X.shape[1]==1907:
            p_val_ver3[i_val, :] = clf.predict(X[i_val])
            p_tst_ver3 += clf.predict(test) / n_class
        elif X.shape[1]==4720:
            p_val_ver4[i_val, :] = clf.predict(X[i_val])
            p_tst_ver4 += clf.predict(test) / n_class
        elif X.shape[1]==4777:
            p_val_ver5[i_val, :] = clf.predict(X[i_val])
            p_tst_ver5 += clf.predict(test) / n_class
        else:
            p_val_ver6[i_val, :] = clf.predict(X[i_val])
            p_tst_ver6 += clf.predict(test) / n_class
            
        del clf
        clear_session()
        gc.collect()
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver1 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver1, axis=1)) * 100:8.4f}%')
print(f'lr ver1 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver1):8.4f}')
print(f'lr ver2 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver2, axis=1)) * 100:8.4f}%')
print(f'lr ver2 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver2):8.4f}')
print(f'lr ver3 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver3, axis=1)) * 100:8.4f}%')
print(f'lr ver3 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver3):8.4f}')
print(f'lr ver4 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver4, axis=1)) * 100:8.4f}%')
print(f'lr ver4 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver4):8.4f}')
print(f'lr ver5 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver5, axis=1)) * 100:8.4f}%')
print(f'lr ver5 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver5):8.4f}')
print(f'lr ver6 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver6, axis=1)) * 100:8.4f}%')
print(f'lr ver6 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver6):8.4f}')


Training model for CV #1
Epoch 1/100
86/86 [==============================] - 1s 10ms/step - loss: 1.1075 - val_loss: 0.8189
Epoch 2/100
86/86 [==============================] - 1s 7ms/step - loss: 0.7186 - val_loss: 0.7328
Epoch 3/100
86/86 [==============================] - 1s 7ms/step - loss: 0.6305 - val_loss: 0.7122
Epoch 4/100
86/86 [==============================] - 1s 7ms/step - loss: 0.5849 - val_loss: 0.7078
Epoch 5/100
86/86 [==============================] - 1s 7ms/step - loss: 0.5537 - val_loss: 0.7098
Epoch 6/100
86/86 [==============================] - 1s 7ms/step - loss: 0.5289 - val_loss: 0.7144
Epoch 7/100
86/86 [==============================] - 1s 7ms/step - loss: 0.5073 - val_loss: 0.7204
Epoch 00007: early stopping
Training model for CV #2
Epoch 1/100
86/86 [==============================] - 1s 10ms/step - loss: 1.0732 - val_loss: 0.8145
Epoch 2/100
86/86 [==============================] - 1s 6ms/step - loss: 0.7129 - val_loss: 0.7329
Epoch 3/100
86/86 [==========

Epoch 5/100
86/86 [==============================] - 1s 9ms/step - loss: 0.3927 - val_loss: 0.6135
Epoch 6/100
86/86 [==============================] - 1s 9ms/step - loss: 0.3591 - val_loss: 0.6222
Epoch 00006: early stopping
Training has finished
****************************************************************************************************
Training model for CV #1
Epoch 1/100
86/86 [==============================] - 1s 14ms/step - loss: 0.9538 - val_loss: 0.6959
Epoch 2/100
86/86 [==============================] - 1s 9ms/step - loss: 0.5806 - val_loss: 0.6315
Epoch 3/100
86/86 [==============================] - 1s 9ms/step - loss: 0.4834 - val_loss: 0.6150
Epoch 4/100
86/86 [==============================] - 1s 9ms/step - loss: 0.4280 - val_loss: 0.6269
Epoch 5/100
86/86 [==============================] - 1s 10ms/step - loss: 0.3884 - val_loss: 0.6382
Epoch 6/100
86/86 [==============================] - 1s 9ms/step - loss: 0.3578 - val_loss: 0.6548
Epoch 00006: early stopping
Tr

Epoch 00006: early stopping
Training has finished
****************************************************************************************************
lr ver1 Accuracy (CV):  73.5855%
lr ver1 Log Loss (CV):   0.7056
lr ver2 Accuracy (CV):  73.8862%
lr ver2 Log Loss (CV):   0.7012
lr ver3 Accuracy (CV):  68.4652%
lr ver3 Log Loss (CV):   0.8314
lr ver4 Accuracy (CV):  77.9351%
lr ver4 Log Loss (CV):   0.6073
lr ver5 Accuracy (CV):  78.0572%
lr ver5 Log Loss (CV):   0.6064
lr ver6 Accuracy (CV):  73.2102%
lr ver6 Log Loss (CV):   0.7271


#### 제출 파일 생성 및 기타 파일 생성

In [45]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver1
sub[sub.columns] = p_tst_ver1
sub.to_csv(sub_ver1_file)

# Ver2
sub[sub.columns] = p_tst_ver2
sub.to_csv(sub_ver2_file)

# Ver3
sub[sub.columns] = p_tst_ver3
sub.to_csv(sub_ver3_file)

# Ver4
sub[sub.columns] = p_tst_ver4
sub.to_csv(sub_ver4_file)

# Ver5
sub[sub.columns] = p_tst_ver5
sub.to_csv(sub_ver5_file)

# Ver6
sub[sub.columns] = p_tst_ver6
sub.to_csv(sub_ver6_file)

In [46]:
# p_val 파일 생성 -> oof

# Ver1
np.savetxt(p_val_ver1_file, p_val_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_val_ver2_file, p_val_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_val_ver3_file, p_val_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_val_ver4_file, p_val_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_val_ver5_file, p_val_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_val_ver6_file, p_val_ver6, fmt='%.18f', delimiter=',')

In [47]:
# p_tst 파일 생성 -> test 

# Ver1
np.savetxt(p_tst_ver1_file, p_tst_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_tst_ver2_file, p_tst_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_tst_ver3_file, p_tst_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_tst_ver4_file, p_tst_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_tst_ver5_file, p_tst_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_tst_ver6_file, p_tst_ver6, fmt='%.18f', delimiter=',')

### 2-2. Feature

In [ ]:
algorithm_name = 'mlp'
feature_name = 'feature'
model_name = f'{algorithm_name}_{feature_name}'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_Ver3_file = feature_dir / f'{feature_name}_Ver3.csv'
feature_Ver4_file = feature_dir / f'{feature_name}_Ver4.csv'


mlp_oof_pred_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
mlp_oof_pred_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
mlp_oof_pred_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
mlp_oof_pred_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'


mlp_test_pred_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'
mlp_test_pred_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'
mlp_test_pred_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'
mlp_test_pred_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'


mlp_submission_ver1_file = sub_dir / f'{model_name}_submission_Ver1.csv'
mlp_submission_ver2_file = sub_dir / f'{model_name}_submission_Ver2.csv'
mlp_submission_ver3_file = sub_dir / f'{model_name}_submission_Ver3.csv'
mlp_submission_ver4_file = sub_dir / f'{model_name}_submission_Ver4.csv'

#### Ver 1 데이터 로드

In [7]:
dataset = pd.read_csv(feature_Ver1_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he was almost choking there was so much so muc...,3.0
1,your sister asked for it i suppose,2.0
2,she was engaged one day as she walked in peru...,1.0
3,the captain was in the porch keeping himself c...,4.0
4,have mercy gentlemen odin flung up his hands d...,3.0


In [8]:
# train set
Ver1_X = dataset.loc[dataset[target_col] != -1 , :]
Ver1_X.drop(columns=target_col,inplace=True,axis=1)
Ver1_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver1_y.astype(int)

# test set
Ver1_test = dataset.loc[dataset[target_col] == -1, :]
Ver1_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver1_X.shape, Ver1_y.shape, Ver1_test.shape)

(54879, 1) (54879,) (19617, 1)


#### Ver 2 데이터 로드

In [9]:
dataset = pd.read_csv(feature_Ver2_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged one day walked perusing janes last let...,1.0
3,captain porch keeping carefully way treacherou...,4.0
4,mercy gentlemen odin flung hands dont write an...,3.0


In [10]:
# train set
Ver2_X = dataset.loc[dataset[target_col] != -1 , :]
Ver2_X.drop(columns=target_col,inplace=True,axis=1)
Ver2_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver2_y.astype(int)

# test set
Ver2_test = dataset.loc[dataset[target_col] == -1, :]
Ver2_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver2_X.shape, Ver2_y.shape, Ver2_test.shape)

(54879, 1) (54879,) (19617, 1)


#### Ver3 데이터 로드

In [11]:
dataset = pd.read_csv(feature_Ver3_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged day walked perusing janes last letter ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,mercy gentlemen flung up hands dont write anyw...,3.0


In [12]:
# train set
Ver3_X = dataset.loc[dataset[target_col] != -1 , :]
Ver3_X.drop(columns=target_col,inplace=True,axis=1)
Ver3_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver3_y.astype(int)

# test set
Ver3_test = dataset.loc[dataset[target_col] == -1, :]
Ver3_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver3_X.shape, Ver3_y.shape, Ver3_test.shape)

(54879, 1) (54879,) (19617, 1)


#### Ver4 데이터 로드

In [13]:
dataset = pd.read_csv(feature_Ver4_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he almost choking there so much so much he wan...,3.0
1,sister asked for it suppose,2.0
2,she engaged one day she walked perusing janes ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,have mercy gentlemen odin flung up hands dont ...,3.0


In [14]:
# train set
Ver4_X = dataset.loc[dataset[target_col] != -1 , :]
Ver4_X.drop(columns=target_col,inplace=True,axis=1)
Ver4_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver4_y.astype(int)

# test set
Ver4_test = dataset.loc[dataset[target_col] == -1, :]
Ver4_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver4_X.shape, Ver4_y.shape, Ver4_test.shape)

(54879, 1) (54879,) (19617, 1)


#### 모델 학습

In [15]:
#파라미터 설정
vocab_size = 30000
embedding_dim = 128
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [16]:
def get_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
    ])
    
    # compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=.0003))
    return model

In [17]:
datasets = [ (Ver1_X, Ver1_test, Ver1_y), (Ver2_X, Ver2_test, Ver2_y),
            (Ver3_X, Ver3_test, Ver3_y), (Ver4_X, Ver4_test, Ver3_y)]

mlp_oof_preds = []
mlp_test_preds = []

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

for number ,(X, test, y) in enumerate(datasets,1):
    print(f'start : {number}')
    # 토큰화
    X_train = np.array([x for x in X['text']])
    X_test = np.array([x for x in test['text']])
    y = np.array(y.values)
    
    tokenizer = Tokenizer(num_words = vocab_size)
    tokenizer.fit_on_texts(X_train)
    word_index = tokenizer.word_index
    
    # 시퀸스화 + 패딩
    train_sequences = tokenizer.texts_to_sequences(X_train)
    test_sequences = tokenizer.texts_to_sequences(X_test)
    trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
    tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
    print(trn.shape, tst.shape)
    
    # oof , test 저장
    mlp_oof_pred = np.zeros((trn.shape[0], n_class))
    mlp_test_pred = np.zeros((tst.shape[0], n_class))
    
    for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
        print(f'traing model for CV #{i}')
        clf = get_model()
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=200,
            batch_size=1024,
            callbacks=[es])
        
        mlp_oof_pred[i_val, :] = clf.predict(trn[i_val])
        mlp_test_pred += clf.predict(tst) / n_fold
        
        del clf
        clear_session()
        gc.collect()
    
    mlp_oof_preds.append(mlp_oof_pred)
    mlp_test_preds.append(mlp_test_pred)
        
    print(f'end : {number}')

start : 1
(54879, 500) (19617, 500)
traing model for CV #1
Epoch 1/200
43/43 [==============================] - 5s 122ms/step - loss: 1.5828 - val_loss: 1.5682
Epoch 2/200
43/43 [==============================] - 5s 115ms/step - loss: 1.5682 - val_loss: 1.5671
Epoch 3/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5665 - val_loss: 1.5652
Epoch 4/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5644 - val_loss: 1.5617
Epoch 5/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5585 - val_loss: 1.5529
Epoch 6/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5439 - val_loss: 1.5304
Epoch 7/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5050 - val_loss: 1.4714
Epoch 8/200
43/43 [==============================] - 5s 115ms/step - loss: 1.4290 - val_loss: 1.3842
Epoch 9/200
43/43 [==============================] - 5s 115ms/step - loss: 1.3373 - val_loss: 1.2935
Epoch 10/200
43/43 [============

43/43 [==============================] - 5s 113ms/step - loss: 0.6514 - val_loss: 0.8110
Epoch 30/200
43/43 [==============================] - 5s 114ms/step - loss: 0.6270 - val_loss: 0.7981
Epoch 31/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6082 - val_loss: 0.7903
Epoch 32/200
43/43 [==============================] - 5s 113ms/step - loss: 0.5889 - val_loss: 0.7910
Epoch 33/200
43/43 [==============================] - 5s 113ms/step - loss: 0.5702 - val_loss: 0.7749
Epoch 34/200
43/43 [==============================] - 5s 114ms/step - loss: 0.5530 - val_loss: 0.7649
Epoch 35/200
43/43 [==============================] - 5s 113ms/step - loss: 0.5353 - val_loss: 0.7597
Epoch 36/200
43/43 [==============================] - 5s 113ms/step - loss: 0.5207 - val_loss: 0.7670
Epoch 37/200
43/43 [==============================] - 5s 114ms/step - loss: 0.5056 - val_loss: 0.7523
Epoch 38/200
43/43 [==============================] - 5s 114ms/step - loss: 0.4939 - val_loss: 

Epoch 17/200
43/43 [==============================] - 5s 113ms/step - loss: 1.0962 - val_loss: 1.1141
Epoch 18/200
43/43 [==============================] - 5s 114ms/step - loss: 1.0646 - val_loss: 1.0908
Epoch 19/200
43/43 [==============================] - 5s 113ms/step - loss: 1.0322 - val_loss: 1.0705
Epoch 20/200
43/43 [==============================] - 5s 114ms/step - loss: 1.0053 - val_loss: 1.0528
Epoch 21/200
43/43 [==============================] - 5s 113ms/step - loss: 0.9842 - val_loss: 1.0387
Epoch 22/200
43/43 [==============================] - 5s 113ms/step - loss: 0.9611 - val_loss: 1.0362
Epoch 23/200
43/43 [==============================] - 5s 114ms/step - loss: 0.9396 - val_loss: 1.0163
Epoch 24/200
43/43 [==============================] - 5s 112ms/step - loss: 0.9169 - val_loss: 1.0112
Epoch 25/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8963 - val_loss: 0.9979
Epoch 26/200
43/43 [==============================] - 5s 114ms/step - loss: 0.8742

43/43 [==============================] - 5s 114ms/step - loss: 0.7075 - val_loss: 0.9087
Epoch 33/200
43/43 [==============================] - 5s 112ms/step - loss: 0.6885 - val_loss: 0.9057
Epoch 34/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6736 - val_loss: 0.9101
Epoch 35/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6593 - val_loss: 0.8999
Epoch 36/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6449 - val_loss: 0.8944
Epoch 37/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6341 - val_loss: 0.9047
Epoch 38/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6226 - val_loss: 0.8921
Epoch 39/200
43/43 [==============================] - 5s 114ms/step - loss: 0.6054 - val_loss: 0.8925
Epoch 40/200
43/43 [==============================] - 5s 112ms/step - loss: 0.5979 - val_loss: 0.8940
Epoch 41/200
43/43 [==============================] - 5s 114ms/step - loss: 0.5826 - val_loss: 

Epoch 22/200
43/43 [==============================] - 5s 113ms/step - loss: 0.9978 - val_loss: 1.0579
Epoch 23/200
43/43 [==============================] - 5s 112ms/step - loss: 0.9708 - val_loss: 1.0348
Epoch 24/200
43/43 [==============================] - 5s 113ms/step - loss: 0.9428 - val_loss: 1.0171
Epoch 25/200
43/43 [==============================] - 5s 112ms/step - loss: 0.9197 - val_loss: 1.0070
Epoch 26/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8950 - val_loss: 0.9870
Epoch 27/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8692 - val_loss: 0.9674
Epoch 28/200
43/43 [==============================] - 5s 112ms/step - loss: 0.8383 - val_loss: 0.9527
Epoch 29/200
43/43 [==============================] - 5s 114ms/step - loss: 0.8131 - val_loss: 0.9340
Epoch 30/200
43/43 [==============================] - 5s 114ms/step - loss: 0.7821 - val_loss: 0.9117
Epoch 31/200
43/43 [==============================] - 5s 114ms/step - loss: 0.7550

43/43 [==============================] - 5s 112ms/step - loss: 1.5691 - val_loss: 1.5687
Epoch 4/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5684 - val_loss: 1.5680
Epoch 5/200
43/43 [==============================] - 5s 113ms/step - loss: 1.5677 - val_loss: 1.5668
Epoch 6/200
43/43 [==============================] - 5s 112ms/step - loss: 1.5659 - val_loss: 1.5649
Epoch 7/200
43/43 [==============================] - 5s 112ms/step - loss: 1.5628 - val_loss: 1.5605
Epoch 8/200
43/43 [==============================] - 5s 113ms/step - loss: 1.5544 - val_loss: 1.5474
Epoch 9/200
43/43 [==============================] - 5s 113ms/step - loss: 1.5332 - val_loss: 1.5166
Epoch 10/200
43/43 [==============================] - 5s 113ms/step - loss: 1.4862 - val_loss: 1.4572
Epoch 11/200
43/43 [==============================] - 5s 112ms/step - loss: 1.4169 - val_loss: 1.3919
Epoch 12/200
43/43 [==============================] - 5s 113ms/step - loss: 1.3545 - val_loss: 1.3462

43/43 [==============================] - 5s 114ms/step - loss: 0.9407 - val_loss: 0.9969
Epoch 20/200
43/43 [==============================] - 5s 114ms/step - loss: 0.8992 - val_loss: 0.9813
Epoch 21/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8678 - val_loss: 0.9558
Epoch 22/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8381 - val_loss: 0.9393
Epoch 23/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8099 - val_loss: 0.9269
Epoch 24/200
43/43 [==============================] - 5s 114ms/step - loss: 0.7830 - val_loss: 0.9087
Epoch 25/200
43/43 [==============================] - 5s 113ms/step - loss: 0.7576 - val_loss: 0.8948
Epoch 26/200
43/43 [==============================] - 5s 114ms/step - loss: 0.7366 - val_loss: 0.8857
Epoch 27/200
43/43 [==============================] - 5s 115ms/step - loss: 0.7092 - val_loss: 0.8732
Epoch 28/200
43/43 [==============================] - 5s 114ms/step - loss: 0.6853 - val_loss: 

In [18]:
for i,j in enumerate(mlp_oof_preds,1):
    print(f'ver{i} logloss = {log_loss(pd.get_dummies(y),j):8.4f}')
    print(f'ver{i} accuracy = {accuracy_score(y, np.argmax(j,axis=1))*100:8.4f}')

ver1 logloss =   0.7848
ver1 logloss =  71.0909
ver2 logloss =   0.8537
ver2 logloss =  68.8661
ver3 logloss =   0.8700
ver3 logloss =  67.3828
ver4 logloss =   0.8009
ver4 logloss =  70.7885


#### 제출 파일 및 기타 파일 생성

In [19]:
# submission 파일 생성

sub = pd.read_csv(sample_file,index_col=0)

# Ver1 
sub[sub.columns] = mlp_test_preds[0]
sub.to_csv(mlp_submission_ver1_file)

# Ver2
sub[sub.columns] = mlp_test_preds[1]
sub.to_csv(mlp_submission_ver2_file)

# Ver3
sub[sub.columns] = mlp_test_preds[2]
sub.to_csv(mlp_submission_ver3_file)
           
# Ver4
sub[sub.columns] = mlp_test_preds[3]
sub.to_csv(mlp_submission_ver4_file)

In [20]:
# mlp_oof_pred 파일 생성

# Ver1
np.savetxt(mlp_oof_pred_ver1_file, mlp_oof_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(mlp_oof_pred_ver2_file, mlp_oof_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(mlp_oof_pred_ver3_file, mlp_oof_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(mlp_oof_pred_ver4_file, mlp_oof_preds[3],fmt='%.18f', delimiter=',')

In [21]:
# mlp_test_pred 파일 생성

# Ver1
np.savetxt(mlp_test_pred_ver1_file, mlp_test_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(mlp_test_pred_ver2_file, mlp_test_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(mlp_test_pred_ver3_file, mlp_test_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(mlp_test_pred_ver4_file, mlp_test_preds[3],fmt='%.18f', delimiter=',')

### 2-3. hashing

In [ ]:
algo_name = 'mlp'
feature_name = 'hashing'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'

p_val_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'

p_val_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
p_tst_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'

p_val_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
p_tst_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'

p_val_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'
p_tst_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'

p_val_ver5_file = val_dir / f'{model_name}_oof_pred_ver5.csv'
p_tst_ver5_file = tst_dir / f'{model_name}_test_pred_ver5.csv'

p_val_ver6_file = val_dir / f'{model_name}_oof_pred_ver6.csv'
p_tst_ver6_file = tst_dir / f'{model_name}_test_pred_ver6.csv'

sub_ver1_file = sub_dir / f'{model_name}_ver1.csv'
sub_ver2_file = sub_dir / f'{model_name}_ver2.csv'
sub_ver3_file = sub_dir / f'{model_name}_ver3.csv'
sub_ver4_file = sub_dir / f'{model_name}_ver4.csv'
sub_ver5_file = sub_dir / f'{model_name}_ver5.csv'
sub_ver6_file = sub_dir / f'{model_name}_ver6.csv'

#### Hashing 피쳐 생성

- nltk의 word_tokenize 사용

In [27]:
vec = HashingVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), n_features=2**10)
X_1 = vec.fit_transform(trn['text']).toarray()
X_tst_1 = vec.transform(tst['text']).toarray()
print(X_1.shape, X_tst_1.shape)

(54879, 1024) (19617, 1024)


In [28]:
X_1[0, :50]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.09950372,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.09950372,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.09950372,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

- nltk의 WordPunctTokenizer 사용

In [29]:
vec = HashingVectorizer(tokenizer=wordPunctTokenizer.tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), n_features=2**10)
X_2 = vec.fit_transform(trn['text']).toarray()
X_tst_2 = vec.transform(tst['text']).toarray()
print(X_2.shape, X_tst_2.shape)

(54879, 1024) (19617, 1024)


In [30]:
X_2[0, :50]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.09950372,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.09950372,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.09950372,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

- keras의 text_to_word_sequence 사용

In [31]:
vec = HashingVectorizer(tokenizer=text_to_word_sequence, stop_words=stopwords.words('english'), ngram_range=(1, 3), n_features=2**10)
X_3 = vec.fit_transform(trn['text']).toarray()
X_tst_3 = vec.transform(tst['text']).toarray()
print(X_3.shape, X_tst_3.shape)

(54879, 1024) (19617, 1024)


In [32]:
X_3[0, :50]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.13245324,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.13245324,
        0.        ,  0.        ,  0.        ,  0.        , -0.13245324,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

- nltk의 word_tokenize 사용, stopword 제거

In [33]:
vec = HashingVectorizer(tokenizer=word_tokenize, ngram_range=(1, 3), n_features=2**10)
X_4 = vec.fit_transform(trn['text']).toarray()
X_tst_4 = vec.transform(tst['text']).toarray()
print(X_4.shape, X_tst_4.shape)

(54879, 1024) (19617, 1024)


In [34]:
X_4[0, :50]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.06917145,  0.        ,
        0.        ,  0.06917145,  0.        , -0.06917145, -0.06917145,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.06917145,
        0.        , -0.06917145,  0.        ,  0.        ,  0.06917145,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.06917145,  0.        ,  0.06917145,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

- nltk의 WordPunctTokenizer 사용, stopword 제거

In [35]:
vec = HashingVectorizer(tokenizer=wordPunctTokenizer.tokenize, ngram_range=(1, 3), n_features=2**10)
X_5 = vec.fit_transform(trn['text']).toarray()
X_tst_5 = vec.transform(tst['text']).toarray()
print(X_5.shape, X_tst_5.shape)

(54879, 1024) (19617, 1024)


In [36]:
X_5[0, :50]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.06917145,  0.        ,
        0.        ,  0.06917145,  0.        , -0.06917145, -0.06917145,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.06917145,
        0.        , -0.06917145,  0.        ,  0.        ,  0.06917145,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.06917145,  0.        ,  0.06917145,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

- keras의 text_to_word_sequence 사용, stopword 제거

In [37]:
vec = HashingVectorizer(tokenizer=text_to_word_sequence, ngram_range=(1, 3), n_features=2**10)
X_6 = vec.fit_transform(trn['text']).toarray()
X_tst_6 = vec.transform(tst['text']).toarray()
print(X_6.shape, X_tst_6.shape)

(54879, 1024) (19617, 1024)


In [38]:
X_6[0, :50]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.07881104,  0.        , -0.07881104,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.07881104,
        0.        , -0.07881104,  0.        ,  0.        ,  0.07881104,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.07881104,  0.        ,  0.07881104,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

#### mlp 모델 학습

In [39]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [40]:
def get_model(number):
    inputs = Input(shape=(number,))
    x = Dense(128, activation='relu')(inputs)
    outputs = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

In [41]:
y = trn.author.values
y.shape

(54879,)

In [42]:
p_val_ver1 = np.zeros((X_1.shape[0], n_class))
p_tst_ver1 = np.zeros((X_tst_1.shape[0], n_class))
p_val_ver2 = np.zeros((X_2.shape[0], n_class))
p_tst_ver2 = np.zeros((X_tst_2.shape[0], n_class))
p_val_ver3 = np.zeros((X_3.shape[0], n_class))
p_tst_ver3 = np.zeros((X_tst_3.shape[0], n_class))
p_val_ver4 = np.zeros((X_4.shape[0], n_class))
p_tst_ver4 = np.zeros((X_tst_4.shape[0], n_class))
p_val_ver5 = np.zeros((X_5.shape[0], n_class))
p_tst_ver5 = np.zeros((X_tst_5.shape[0], n_class))
p_val_ver6 = np.zeros((X_6.shape[0], n_class))
p_tst_ver6 = np.zeros((X_tst_6.shape[0], n_class))

for number, (X, test) in enumerate([(X_1, X_tst_1), (X_2, X_tst_2), (X_3, X_tst_3),
               (X_4, X_tst_4), (X_5, X_tst_5), (X_6, X_tst_6)],1):
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf = get_model(X.shape[1])
        clf.fit(X[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(X[i_val], to_categorical(y[i_val])),
            epochs=100,
            batch_size=512,
            callbacks=[es])
       
        # Predict
        if number==1:
            p_val_ver1[i_val, :] = clf.predict(X[i_val])
            p_tst_ver1 += clf.predict(test) / n_class
        elif number==2:
            p_val_ver2[i_val, :] = clf.predict(X[i_val])
            p_tst_ver2 += clf.predict(test) / n_class
        elif number==3:
            p_val_ver3[i_val, :] = clf.predict(X[i_val])
            p_tst_ver3 += clf.predict(test) / n_class
        elif number==4:
            p_val_ver4[i_val, :] = clf.predict(X[i_val])
            p_tst_ver4 += clf.predict(test) / n_class
        elif number==5:
            p_val_ver5[i_val, :] = clf.predict(X[i_val])
            p_tst_ver5 += clf.predict(test) / n_class
        else:
            p_val_ver6[i_val, :] = clf.predict(X[i_val])
            p_tst_ver6 += clf.predict(test) / n_class
            
        del clf
        clear_session()
        gc.collect()
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver1 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver1, axis=1)) * 100:8.4f}%')
print(f'lr ver1 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver1):8.4f}')
print(f'lr ver2 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver2, axis=1)) * 100:8.4f}%')
print(f'lr ver2 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver2):8.4f}')
print(f'lr ver3 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver3, axis=1)) * 100:8.4f}%')
print(f'lr ver3 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver3):8.4f}')
print(f'lr ver4 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver4, axis=1)) * 100:8.4f}%')
print(f'lr ver4 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver4):8.4f}')
print(f'lr ver5 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver5, axis=1)) * 100:8.4f}%')
print(f'lr ver5 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver5):8.4f}')
print(f'lr ver6 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver6, axis=1)) * 100:8.4f}%')
print(f'lr ver6 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver6):8.4f}')

Training model for CV #1
Epoch 1/100
86/86 [==============================] - 1s 6ms/step - loss: 1.4758 - val_loss: 1.3263
Epoch 2/100
86/86 [==============================] - 0s 4ms/step - loss: 1.2150 - val_loss: 1.1631
Epoch 3/100
86/86 [==============================] - 0s 4ms/step - loss: 1.1022 - val_loss: 1.1208
Epoch 4/100
86/86 [==============================] - 0s 4ms/step - loss: 1.0626 - val_loss: 1.1094
Epoch 5/100
86/86 [==============================] - 0s 4ms/step - loss: 1.0423 - val_loss: 1.1042
Epoch 6/100
86/86 [==============================] - 0s 4ms/step - loss: 1.0282 - val_loss: 1.1022
Epoch 7/100
86/86 [==============================] - 0s 4ms/step - loss: 1.0172 - val_loss: 1.0973
Epoch 8/100
86/86 [==============================] - 0s 4ms/step - loss: 1.0056 - val_loss: 1.0929
Epoch 9/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9931 - val_loss: 1.0899
Epoch 10/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9817 - v

Epoch 19/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9049 - val_loss: 1.0926
Epoch 20/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8904 - val_loss: 1.0912
Epoch 21/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8752 - val_loss: 1.0896
Epoch 22/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8593 - val_loss: 1.0880
Epoch 23/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8432 - val_loss: 1.0860
Epoch 24/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8265 - val_loss: 1.0868
Epoch 25/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8097 - val_loss: 1.0874
Epoch 26/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7914 - val_loss: 1.0852
Epoch 00026: early stopping
Training model for CV #2
Epoch 1/100
86/86 [==============================] - 1s 6ms/step - loss: 1.4938 - val_loss: 1.3482
Epoch 2/100
86/86 [=============================

86/86 [==============================] - 0s 4ms/step - loss: 0.9005 - val_loss: 1.0950
Epoch 19/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8862 - val_loss: 1.0925
Epoch 20/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8708 - val_loss: 1.0917
Epoch 21/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8561 - val_loss: 1.0911
Epoch 22/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8402 - val_loss: 1.0899
Epoch 23/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8243 - val_loss: 1.0890
Epoch 24/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8077 - val_loss: 1.0893
Epoch 25/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7907 - val_loss: 1.0897
Epoch 00025: early stopping
Training model for CV #5
Epoch 1/100
86/86 [==============================] - 1s 6ms/step - loss: 1.4829 - val_loss: 1.3410
Epoch 2/100
86/86 [==============================] - 0s 4ms/s

86/86 [==============================] - 0s 4ms/step - loss: 1.0324 - val_loss: 1.0593
Epoch 4/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9939 - val_loss: 1.0498
Epoch 5/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9754 - val_loss: 1.0448
Epoch 6/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9638 - val_loss: 1.0432
Epoch 7/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9530 - val_loss: 1.0404
Epoch 8/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9426 - val_loss: 1.0350
Epoch 9/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9332 - val_loss: 1.0315
Epoch 10/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9229 - val_loss: 1.0278
Epoch 11/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9123 - val_loss: 1.0236
Epoch 12/100
86/86 [==============================] - 0s 4ms/step - loss: 0.9017 - val_loss: 1.0199
Epoch 13/100
86/86 

Epoch 17/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8255 - val_loss: 0.9834
Epoch 18/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8117 - val_loss: 0.9789
Epoch 19/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7977 - val_loss: 0.9772
Epoch 20/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7839 - val_loss: 0.9754
Epoch 21/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7696 - val_loss: 0.9743
Epoch 22/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7549 - val_loss: 0.9708
Epoch 23/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7405 - val_loss: 0.9696
Epoch 24/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7254 - val_loss: 0.9694
Epoch 25/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7107 - val_loss: 0.9690
Epoch 26/100
86/86 [==============================] - 0s 4ms/step - loss: 0.6956 - val_loss: 0.9688


Epoch 16/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8800 - val_loss: 1.0294
Epoch 17/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8674 - val_loss: 1.0272
Epoch 18/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8541 - val_loss: 1.0252
Epoch 19/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8408 - val_loss: 1.0231
Epoch 20/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8269 - val_loss: 1.0199
Epoch 21/100
86/86 [==============================] - 0s 4ms/step - loss: 0.8125 - val_loss: 1.0174
Epoch 22/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7968 - val_loss: 1.0138
Epoch 23/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7810 - val_loss: 1.0125
Epoch 24/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7649 - val_loss: 1.0097
Epoch 25/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7483 - val_loss: 1.0076


Epoch 00027: early stopping
Training model for CV #2
Epoch 1/100
86/86 [==============================] - 1s 6ms/step - loss: 1.4908 - val_loss: 1.3592
Epoch 2/100
86/86 [==============================] - 0s 4ms/step - loss: 1.2432 - val_loss: 1.2168
Epoch 3/100
86/86 [==============================] - 0s 4ms/step - loss: 1.1445 - val_loss: 1.1876
Epoch 4/100
86/86 [==============================] - 0s 4ms/step - loss: 1.1118 - val_loss: 1.1818
Epoch 5/100
86/86 [==============================] - 0s 4ms/step - loss: 1.0965 - val_loss: 1.1811
Epoch 6/100
86/86 [==============================] - 0s 4ms/step - loss: 1.0865 - val_loss: 1.1822
Epoch 7/100
86/86 [==============================] - 0s 4ms/step - loss: 1.0774 - val_loss: 1.1790
Epoch 8/100
86/86 [==============================] - 0s 4ms/step - loss: 1.0682 - val_loss: 1.1776
Epoch 9/100
86/86 [==============================] - 0s 4ms/step - loss: 1.0585 - val_loss: 1.1750
Epoch 10/100
86/86 [==============================] - 0s

#### 제출 파일 생성 및 기타 파일 생성

In [43]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver1
sub[sub.columns] = p_tst_ver1
sub.to_csv(sub_ver1_file)

# Ver2
sub[sub.columns] = p_tst_ver2
sub.to_csv(sub_ver2_file)

# Ver3
sub[sub.columns] = p_tst_ver3
sub.to_csv(sub_ver3_file)

# Ver4
sub[sub.columns] = p_tst_ver4
sub.to_csv(sub_ver4_file)

# Ver5
sub[sub.columns] = p_tst_ver5
sub.to_csv(sub_ver5_file)

# Ver6
sub[sub.columns] = p_tst_ver6
sub.to_csv(sub_ver6_file)

In [44]:
# p_val 파일 생성 -> oof

# Ver1
np.savetxt(p_val_ver1_file, p_val_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_val_ver2_file, p_val_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_val_ver3_file, p_val_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_val_ver4_file, p_val_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_val_ver5_file, p_val_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_val_ver6_file, p_val_ver6, fmt='%.18f', delimiter=',')

In [45]:
# p_tst 파일 생성 -> test 

# Ver1
np.savetxt(p_tst_ver1_file, p_tst_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_tst_ver2_file, p_tst_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_tst_ver3_file, p_tst_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_tst_ver4_file, p_tst_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_tst_ver5_file, p_tst_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_tst_ver6_file, p_tst_ver6, fmt='%.18f', delimiter=',')

### 2-4 tf-idf

In [ ]:
algo_name = 'mlp'
feature_name = 'tfidf'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'

p_val_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'

p_val_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
p_tst_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'

p_val_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
p_tst_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'

p_val_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'
p_tst_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'

p_val_ver5_file = val_dir / f'{model_name}_oof_pred_ver5.csv'
p_tst_ver5_file = tst_dir / f'{model_name}_test_pred_ver5.csv'

p_val_ver6_file = val_dir / f'{model_name}_oof_pred_ver6.csv'
p_tst_ver6_file = tst_dir / f'{model_name}_test_pred_ver6.csv'

sub_ver1_file = sub_dir / f'{model_name}_ver1.csv'
sub_ver2_file = sub_dir / f'{model_name}_ver2.csv'
sub_ver3_file = sub_dir / f'{model_name}_ver3.csv'
sub_ver4_file = sub_dir / f'{model_name}_ver4.csv'
sub_ver5_file = sub_dir / f'{model_name}_ver5.csv'
sub_ver6_file = sub_dir / f'{model_name}_ver6.csv'

#### TF-IDF 피쳐 생성

- nltk의 word_tokenize 사용

In [27]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_1 = vec.fit_transform(trn['text']).toarray()
X_tst_1 = vec.transform(tst['text']).toarray()
print(X_1.shape, X_tst_1.shape)

(54879, 5897) (19617, 5897)


In [28]:
X_1[0, :50]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

- nltk의 WordPunctTokenizer 사용

In [29]:
vec = TfidfVectorizer(tokenizer=wordPunctTokenizer.tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_2 = vec.fit_transform(trn['text']).toarray()
X_tst_2 = vec.transform(tst['text']).toarray()
print(X_2.shape, X_tst_2.shape)

(54879, 5772) (19617, 5772)


In [30]:
X_2[0, :50]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

- keras의 text_to_word_sequence 사용

In [31]:
vec = TfidfVectorizer(tokenizer=text_to_word_sequence, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_3 = vec.fit_transform(trn['text']).toarray()
X_tst_3 = vec.transform(tst['text']).toarray()
print(X_3.shape, X_tst_3.shape)

(54879, 3745) (19617, 3745)


In [32]:
X_3[0, :50]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

- nltk의 word_tokenize 사용, stopword 제거

In [33]:
vec = TfidfVectorizer(tokenizer=word_tokenize, ngram_range=(1, 3), min_df=50)
X_4 = vec.fit_transform(trn['text']).toarray()
X_tst_4 = vec.transform(tst['text']).toarray()
print(X_4.shape, X_tst_4.shape)

(54879, 12254) (19617, 12254)


In [34]:
X_4[0, :50]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

- nltk의 WordPunctTokenizer 사용, stopword 제거

In [35]:
vec = TfidfVectorizer(tokenizer=wordPunctTokenizer.tokenize, ngram_range=(1, 3), min_df=50)
X_5 = vec.fit_transform(trn['text']).toarray()
X_tst_5 = vec.transform(tst['text']).toarray()
print(X_5.shape, X_tst_5.shape)

(54879, 12267) (19617, 12267)


In [36]:
X_5[0, :50]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

- keras의 text_to_word_sequence 사용, stopword 제거

In [37]:
vec = TfidfVectorizer(tokenizer=text_to_word_sequence, ngram_range=(1, 3), min_df=50)
X_6 = vec.fit_transform(trn['text']).toarray()
X_tst_6 = vec.transform(tst['text']).toarray()
print(X_6.shape, X_tst_6.shape)

(54879, 9555) (19617, 9555)


In [38]:
X_6[0, :50]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

#### mlp 모델 학습

In [39]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [40]:
def get_model(number):
    inputs = Input(shape=(number,))
    x = Dense(128, activation='relu')(inputs)
    outputs = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

In [41]:
y = trn.author.values
y.shape

(54879,)

In [42]:
p_val_ver1 = np.zeros((X_1.shape[0], n_class))
p_tst_ver1 = np.zeros((X_tst_1.shape[0], n_class))
p_val_ver2 = np.zeros((X_2.shape[0], n_class))
p_tst_ver2 = np.zeros((X_tst_2.shape[0], n_class))
p_val_ver3 = np.zeros((X_3.shape[0], n_class))
p_tst_ver3 = np.zeros((X_tst_3.shape[0], n_class))
p_val_ver4 = np.zeros((X_4.shape[0], n_class))
p_tst_ver4 = np.zeros((X_tst_4.shape[0], n_class))
p_val_ver5 = np.zeros((X_5.shape[0], n_class))
p_tst_ver5 = np.zeros((X_tst_5.shape[0], n_class))
p_val_ver6 = np.zeros((X_6.shape[0], n_class))
p_tst_ver6 = np.zeros((X_tst_6.shape[0], n_class))

for X, test in [(X_1, X_tst_1), (X_2, X_tst_2), (X_3, X_tst_3),
               (X_4, X_tst_4), (X_5, X_tst_5), (X_6, X_tst_6)]:  
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf = get_model(X.shape[1])
        clf.fit(X[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(X[i_val], to_categorical(y[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[es])
       
        # Predict
        if X.shape[1]==5897:
            p_val_ver1[i_val, :] = clf.predict(X[i_val])
            p_tst_ver1 += clf.predict(test) / n_class
        elif X.shape[1]==5772:
            p_val_ver2[i_val, :] = clf.predict(X[i_val])
            p_tst_ver2 += clf.predict(test) / n_class
        elif X.shape[1]==3745:
            p_val_ver3[i_val, :] = clf.predict(X[i_val])
            p_tst_ver3 += clf.predict(test) / n_class
        elif X.shape[1]==12254:
            p_val_ver4[i_val, :] = clf.predict(X[i_val])
            p_tst_ver4 += clf.predict(test) / n_class
        elif X.shape[1]==12267:
            p_val_ver5[i_val, :] = clf.predict(X[i_val])
            p_tst_ver5 += clf.predict(test) / n_class
        else:
            p_val_ver6[i_val, :] = clf.predict(X[i_val])
            p_tst_ver6 += clf.predict(test) / n_class
            
        del clf
        clear_session()
        gc.collect()
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver1 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver1, axis=1)) * 100:8.4f}%')
print(f'lr ver1 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver1):8.4f}')
print(f'lr ver2 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver2, axis=1)) * 100:8.4f}%')
print(f'lr ver2 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver2):8.4f}')
print(f'lr ver3 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver3, axis=1)) * 100:8.4f}%')
print(f'lr ver3 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver3):8.4f}')
print(f'lr ver4 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver4, axis=1)) * 100:8.4f}%')
print(f'lr ver4 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver4):8.4f}')
print(f'lr ver5 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver5, axis=1)) * 100:8.4f}%')
print(f'lr ver5 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver5):8.4f}')
print(f'lr ver6 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver6, axis=1)) * 100:8.4f}%')
print(f'lr ver6 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver6):8.4f}')

Training model for CV #1
Epoch 1/10
86/86 [==============================] - 1s 12ms/step - loss: 1.3143 - val_loss: 0.9899
Epoch 2/10
86/86 [==============================] - 1s 7ms/step - loss: 0.7924 - val_loss: 0.7260
Epoch 3/10
86/86 [==============================] - 1s 7ms/step - loss: 0.6066 - val_loss: 0.6562
Epoch 4/10
86/86 [==============================] - 1s 7ms/step - loss: 0.5257 - val_loss: 0.6337
Epoch 5/10
86/86 [==============================] - 1s 7ms/step - loss: 0.4771 - val_loss: 0.6309
Epoch 6/10
86/86 [==============================] - 1s 7ms/step - loss: 0.4423 - val_loss: 0.6324
Epoch 7/10
86/86 [==============================] - 1s 7ms/step - loss: 0.4155 - val_loss: 0.6426
Epoch 8/10
86/86 [==============================] - 1s 7ms/step - loss: 0.3939 - val_loss: 0.6538
Epoch 00008: early stopping
Training model for CV #2
Epoch 1/10
86/86 [==============================] - 1s 12ms/step - loss: 1.3105 - val_loss: 0.9958
Epoch 2/10
86/86 [====================

Epoch 5/10
86/86 [==============================] - 1s 7ms/step - loss: 0.4723 - val_loss: 0.6284
Epoch 6/10
86/86 [==============================] - 1s 7ms/step - loss: 0.4378 - val_loss: 0.6325
Epoch 7/10
86/86 [==============================] - 1s 7ms/step - loss: 0.4120 - val_loss: 0.6404
Epoch 8/10
86/86 [==============================] - 1s 7ms/step - loss: 0.3903 - val_loss: 0.6530
Epoch 00008: early stopping
Training model for CV #5
Epoch 1/10
86/86 [==============================] - 1s 11ms/step - loss: 1.2999 - val_loss: 0.9718
Epoch 2/10
86/86 [==============================] - 1s 7ms/step - loss: 0.7790 - val_loss: 0.7183
Epoch 3/10
86/86 [==============================] - 1s 7ms/step - loss: 0.5970 - val_loss: 0.6547
Epoch 4/10
86/86 [==============================] - 1s 7ms/step - loss: 0.5179 - val_loss: 0.6365
Epoch 5/10
86/86 [==============================] - 1s 7ms/step - loss: 0.4697 - val_loss: 0.6343
Epoch 6/10
86/86 [==============================] - 1s 7ms/step 

86/86 [==============================] - 1s 9ms/step - loss: 0.5777 - val_loss: 0.6591
Epoch 4/10
86/86 [==============================] - 1s 9ms/step - loss: 0.4794 - val_loss: 0.6393
Epoch 5/10
86/86 [==============================] - 1s 9ms/step - loss: 0.4166 - val_loss: 0.6385
Epoch 6/10
86/86 [==============================] - 1s 9ms/step - loss: 0.3707 - val_loss: 0.6519
Epoch 7/10
86/86 [==============================] - 1s 9ms/step - loss: 0.3345 - val_loss: 0.6672
Epoch 00007: early stopping
Training model for CV #2
Epoch 1/10
86/86 [==============================] - 1s 16ms/step - loss: 1.3139 - val_loss: 1.0075
Epoch 2/10
86/86 [==============================] - 1s 9ms/step - loss: 0.7898 - val_loss: 0.7400
Epoch 3/10
86/86 [==============================] - 1s 9ms/step - loss: 0.5810 - val_loss: 0.6641
Epoch 4/10
86/86 [==============================] - 1s 9ms/step - loss: 0.4816 - val_loss: 0.6420
Epoch 5/10
86/86 [==============================] - 1s 9ms/step - loss: 0.4

#### 제출 파일 생성 및 기타 파일 생성

In [43]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver1
sub[sub.columns] = p_tst_ver1
sub.to_csv(sub_ver1_file)

# Ver2
sub[sub.columns] = p_tst_ver2
sub.to_csv(sub_ver2_file)

# Ver3
sub[sub.columns] = p_tst_ver3
sub.to_csv(sub_ver3_file)

# Ver4
sub[sub.columns] = p_tst_ver4
sub.to_csv(sub_ver4_file)

# Ver5
sub[sub.columns] = p_tst_ver5
sub.to_csv(sub_ver5_file)

# Ver6
sub[sub.columns] = p_tst_ver6
sub.to_csv(sub_ver6_file)

In [44]:
# p_val 파일 생성 -> oof

# Ver1
np.savetxt(p_val_ver1_file, p_val_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_val_ver2_file, p_val_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_val_ver3_file, p_val_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_val_ver4_file, p_val_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_val_ver5_file, p_val_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_val_ver6_file, p_val_ver6, fmt='%.18f', delimiter=',')

In [45]:
# p_tst 파일 생성 -> test 

# Ver1
np.savetxt(p_tst_ver1_file, p_tst_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_tst_ver2_file, p_tst_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_tst_ver3_file, p_tst_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_tst_ver4_file, p_tst_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_tst_ver5_file, p_tst_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_tst_ver6_file, p_tst_ver6, fmt='%.18f', delimiter=',')

### 2-5. tf-idf-pca

In [ ]:
algo_name = 'mlp'
feature_name = 'tfidf-pca' #학습할 피처- tfidf 피처를 pca한 데이터
feature_target = 'feature_target'

model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv' #학습할 피처 가져오기
feature_target_file = feature_dir / f'{feature_target}.csv' #target 데이터 가져오기

p_val_ver7_file = val_dir / f'{model_name}_oof_pred_ver7.csv'
p_tst_ver7_file = tst_dir / f'{model_name}_test_pred_ver7.csv'
sub_ver7_file = sub_dir / f'{model_name}_ver7.csv'

#### 데이터 셋 불러오기

In [6]:
X = pd.read_csv(feature_file, index_col=0).values
y = pd.read_csv(feature_target_file,index_col=0).values

X_7 = X[:54879]
X_tst_7 = X[54879:]


In [7]:
print(X_7.shape)
print(X_tst_7.shape)
print(type(X_7))

(54879, 500)
(19617, 500)
<class 'numpy.ndarray'>


#### mlp 모델 학습

In [8]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [9]:
def get_model(number):
    inputs = Input(shape=(number,))
    x = Dense(128, activation='relu')(inputs)
    outputs = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

In [10]:
p_val_ver7 = np.zeros((X_7.shape[0], n_class))
p_tst_ver7 = np.zeros((X_tst_7.shape[0], n_class))

for X, test in [(X_7, X_tst_7)]:  
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf = get_model(X.shape[1])
        clf.fit(X[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(X[i_val], to_categorical(y[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[es])
       
        # Predict
        if X.shape[1]==500:
            p_val_ver7[i_val, :] = clf.predict(X[i_val])
            p_tst_ver7 += clf.predict(test) / n_class
            
        del clf
        clear_session()
        gc.collect()
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr Ver7 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver7, axis=1)) * 100:8.4f}%')
print(f'lr Ver7 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver7):8.4f}')


Training model for CV #1
Epoch 1/10
86/86 [==============================] - 1s 7ms/step - loss: 1.4803 - val_loss: 1.2990
Epoch 2/10
86/86 [==============================] - 0s 4ms/step - loss: 1.1005 - val_loss: 0.9587
Epoch 3/10
86/86 [==============================] - 0s 4ms/step - loss: 0.8697 - val_loss: 0.8391
Epoch 4/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7923 - val_loss: 0.7999
Epoch 5/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7626 - val_loss: 0.7845
Epoch 6/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7479 - val_loss: 0.7774
Epoch 7/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7392 - val_loss: 0.7722
Epoch 8/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7332 - val_loss: 0.7698
Epoch 9/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7288 - val_loss: 0.7659
Epoch 10/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7248 - val_loss: 0

Exception: Data must be 1-dimensional

#### 제출 파일 생성 및 기타 파일 생성

In [ ]:
# submission 파일 생성
sub = pd.read_csv(sample_file, index_col=0)
# Ver1
sub[sub.columns] = p_tst_ver7
sub.to_csv(sub_ver7_file)

In [ ]:
# p_val 파일 생성 -> oof
# Ver1
np.savetxt(p_val_ver7_file, p_val_ver7, fmt='%.18f', delimiter=',')

In [ ]:
# p_tst 파일 생성 -> test 
# Ver1
np.savetxt(p_tst_ver7_file, p_tst_ver7, fmt='%.18f', delimiter=',')

## 3. LR

### 3-1. bow

In [ ]:
algo_name = 'lr'
feature_name = 'bow'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'

p_val_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'

p_val_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
p_tst_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'

p_val_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
p_tst_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'

p_val_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'
p_tst_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'

p_val_ver5_file = val_dir / f'{model_name}_oof_pred_ver5.csv'
p_tst_ver5_file = tst_dir / f'{model_name}_test_pred_ver5.csv'

p_val_ver6_file = val_dir / f'{model_name}_oof_pred_ver6.csv'
p_tst_ver6_file = tst_dir / f'{model_name}_test_pred_ver6.csv'

sub_ver1_file = sub_dir / f'{model_name}_ver1.csv'
sub_ver2_file = sub_dir / f'{model_name}_ver2.csv'
sub_ver3_file = sub_dir / f'{model_name}_ver3.csv'
sub_ver4_file = sub_dir / f'{model_name}_ver4.csv'
sub_ver5_file = sub_dir / f'{model_name}_ver5.csv'
sub_ver6_file = sub_dir / f'{model_name}_ver6.csv'

#### DTM 피쳐 생성

- nltk의 word_tokenize 사용

In [27]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_1 = vec.fit_transform(trn['text'])
X_tst_1 = vec.transform(tst['text'])
print(X_1.shape, X_tst_1.shape)

(54879, 2685) (19617, 2685)


In [28]:
X_1[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

- nltk의 WordPunctTokenizer 사용

In [29]:
vec = CountVectorizer(tokenizer=wordPunctTokenizer.tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_2 = vec.fit_transform(trn['text'])
X_tst_2 = vec.transform(tst['text'])
print(X_2.shape, X_tst_2.shape)

(54879, 2655) (19617, 2655)


In [30]:
X_2[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 4, 0, 0, 0]])

- keras의 text_to_word_sequence 사용

In [31]:
vec = CountVectorizer(tokenizer=text_to_word_sequence, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_3 = vec.fit_transform(trn['text'])
X_tst_3 = vec.transform(tst['text'])
print(X_3.shape, X_tst_3.shape)

(54879, 1907) (19617, 1907)


In [32]:
X_3[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

- nltk의 word_tokenize 사용 , stopword 없음

In [33]:
vec = CountVectorizer(tokenizer=word_tokenize, ngram_range=(1, 2), min_df=100)
X_4 = vec.fit_transform(trn['text'])
X_tst_4 = vec.transform(tst['text'])
print(X_4.shape, X_tst_4.shape)

(54879, 4720) (19617, 4720)


In [34]:
X_4[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

- nltk의 WordPunctTokenizer 사용 , stopword 없음

In [35]:
vec = CountVectorizer(tokenizer=wordPunctTokenizer.tokenize, ngram_range=(1, 2), min_df=100)
X_5 = vec.fit_transform(trn['text'])
X_tst_5 = vec.transform(tst['text'])
print(X_5.shape, X_tst_5.shape)

(54879, 4777) (19617, 4777)


In [36]:
X_5[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

- keras의 text_to_word_sequence 사용 , stopword 없음

In [37]:
vec = CountVectorizer(tokenizer=text_to_word_sequence, ngram_range=(1, 2), min_df=100)
X_6 = vec.fit_transform(trn['text'])
X_tst_6 = vec.transform(tst['text'])
print(X_6.shape, X_tst_6.shape)

(54879, 4091) (19617, 4091)


In [38]:
X_6[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

#### 로지스틱회귀 모델 학습

In [40]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [41]:
y = trn.author.values
y.shape

(54879,)

In [43]:
p_val_ver1 = np.zeros((X_1.shape[0], n_class))
p_tst_ver1 = np.zeros((X_tst_1.shape[0], n_class))
p_val_ver2 = np.zeros((X_2.shape[0], n_class))
p_tst_ver2 = np.zeros((X_tst_2.shape[0], n_class))
p_val_ver3 = np.zeros((X_3.shape[0], n_class))
p_tst_ver3 = np.zeros((X_tst_3.shape[0], n_class))
p_val_ver4 = np.zeros((X_4.shape[0], n_class))
p_tst_ver4 = np.zeros((X_tst_4.shape[0], n_class))
p_val_ver5 = np.zeros((X_5.shape[0], n_class))
p_tst_ver5 = np.zeros((X_tst_5.shape[0], n_class))
p_val_ver6 = np.zeros((X_6.shape[0], n_class))
p_tst_ver6 = np.zeros((X_tst_6.shape[0], n_class))

for X, test in [(X_1, X_tst_1), (X_2, X_tst_2), (X_3, X_tst_3),
               (X_4, X_tst_4), (X_5, X_tst_5), (X_6, X_tst_6)]: 
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        X_train, X_val = X[i_trn], X[i_val]
        y_train, y_val = y[i_trn], y[i_val]
        
        lr_clf = LogisticRegression()
        lr_clf.fit(X_train,y_train)
       
        # Predict
        if X.shape[1]==2685:
            p_val_ver1[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver1 += lr_clf.predict_proba(test) / n_class
        elif X.shape[1]==2655:
            p_val_ver2[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver2 += lr_clf.predict_proba(test) / n_class
        elif X.shape[1]==1907:
            p_val_ver3[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver3 += lr_clf.predict_proba(test) / n_class
        elif X.shape[1]==4720:
            p_val_ver4[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver4 += lr_clf.predict_proba(test) / n_class
        elif X.shape[1]==4777:
            p_val_ver5[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver5 += lr_clf.predict_proba(test) / n_class
        else:
            p_val_ver6[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver6 += lr_clf.predict_proba(test) / n_class
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver1 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver1, axis=1)) * 100:8.4f}%')
print(f'lr ver1 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver1):8.4f}')
print(f'lr ver2 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver2, axis=1)) * 100:8.4f}%')
print(f'lr ver2 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver2):8.4f}')
print(f'lr ver3 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver3, axis=1)) * 100:8.4f}%')
print(f'lr ver3 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver3):8.4f}')
print(f'lr ver4 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver4, axis=1)) * 100:8.4f}%')
print(f'lr ver4 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver4):8.4f}')
print(f'lr ver5 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver5, axis=1)) * 100:8.4f}%')
print(f'lr ver5 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver5):8.4f}')
print(f'lr ver6 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver6, axis=1)) * 100:8.4f}%')
print(f'lr ver6 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver6):8.4f}')


Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training

#### 제출 파일 생성 및 기타 파일 생성

In [44]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver1
sub[sub.columns] = p_tst_ver1
sub.to_csv(sub_ver1_file)

# Ver2
sub[sub.columns] = p_tst_ver2
sub.to_csv(sub_ver2_file)

# Ver3
sub[sub.columns] = p_tst_ver3
sub.to_csv(sub_ver3_file)

# Ver4
sub[sub.columns] = p_tst_ver4
sub.to_csv(sub_ver4_file)

# Ver5
sub[sub.columns] = p_tst_ver5
sub.to_csv(sub_ver5_file)

# Ver6
sub[sub.columns] = p_tst_ver6
sub.to_csv(sub_ver6_file)

In [45]:
# p_val 파일 생성 -> oof

# Ver1
np.savetxt(p_val_ver1_file, p_val_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_val_ver2_file, p_val_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_val_ver3_file, p_val_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_val_ver4_file, p_val_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_val_ver5_file, p_val_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_val_ver6_file, p_val_ver6, fmt='%.18f', delimiter=',')

In [46]:
# p_tst 파일 생성 -> test 

# Ver1
np.savetxt(p_tst_ver1_file, p_tst_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_tst_ver2_file, p_tst_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_tst_ver3_file, p_tst_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_tst_ver4_file, p_tst_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_tst_ver5_file, p_tst_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_tst_ver6_file, p_tst_ver6, fmt='%.18f', delimiter=',')

### 3-2. hashing

In [ ]:
algo_name = 'lr'
feature_name = 'hashing'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'

p_val_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'

p_val_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
p_tst_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'

p_val_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
p_tst_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'

p_val_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'
p_tst_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'

p_val_ver5_file = val_dir / f'{model_name}_oof_pred_ver5.csv'
p_tst_ver5_file = tst_dir / f'{model_name}_test_pred_ver5.csv'

p_val_ver6_file = val_dir / f'{model_name}_oof_pred_ver6.csv'
p_tst_ver6_file = tst_dir / f'{model_name}_test_pred_ver6.csv'

sub_ver1_file = sub_dir / f'{model_name}_ver1.csv'
sub_ver2_file = sub_dir / f'{model_name}_ver2.csv'
sub_ver3_file = sub_dir / f'{model_name}_ver3.csv'
sub_ver4_file = sub_dir / f'{model_name}_ver4.csv'
sub_ver5_file = sub_dir / f'{model_name}_ver5.csv'
sub_ver6_file = sub_dir / f'{model_name}_ver6.csv'

#### Hashing 피쳐 생성

- nltk의 word_tokenize 사용

In [27]:
vec = HashingVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), n_features=2**10)
X_1 = vec.fit_transform(trn['text'])
X_tst_1 = vec.transform(tst['text'])
print(X_1.shape, X_tst_1.shape)

(54879, 1024) (19617, 1024)


In [28]:
X_1[0, :50].todense()

matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.09950372,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.09950372,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.09950372,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

- nltk의 WordPunctTokenizer 사용

In [29]:
vec = HashingVectorizer(tokenizer=wordPunctTokenizer.tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), n_features=2**10)
X_2 = vec.fit_transform(trn['text'])
X_tst_2 = vec.transform(tst['text'])
print(X_2.shape, X_tst_2.shape)

(54879, 1024) (19617, 1024)


In [30]:
X_2[0, :50].todense()

matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.09950372,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.09950372,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.09950372,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

- keras의 text_to_word_sequence 사용

In [31]:
vec = HashingVectorizer(tokenizer=text_to_word_sequence, stop_words=stopwords.words('english'), ngram_range=(1, 3), n_features=2**10)
X_3 = vec.fit_transform(trn['text'])
X_tst_3 = vec.transform(tst['text'])
print(X_3.shape, X_tst_3.shape)

(54879, 1024) (19617, 1024)


In [32]:
X_3[0, :50].todense()

matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.13245324,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.13245324,
          0.        ,  0.        ,  0.        ,  0.        , -0.13245324,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

- nltk의 word_tokenize 사용, stopword 제거

In [33]:
vec = HashingVectorizer(tokenizer=word_tokenize, ngram_range=(1, 3), n_features=2**10)
X_4 = vec.fit_transform(trn['text'])
X_tst_4 = vec.transform(tst['text'])
print(X_4.shape, X_tst_4.shape)

(54879, 1024) (19617, 1024)


In [34]:
X_4[0, :50].todense()

matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.06917145,  0.        ,
          0.        ,  0.06917145,  0.        , -0.06917145, -0.06917145,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.06917145,
          0.        , -0.06917145,  0.        ,  0.        ,  0.06917145,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.06917145,  0.        ,  0.06917145,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

- nltk의 WordPunctTokenizer 사용, stopword 제거

In [35]:
vec = HashingVectorizer(tokenizer=wordPunctTokenizer.tokenize, ngram_range=(1, 3), n_features=2**10)
X_5 = vec.fit_transform(trn['text'])
X_tst_5 = vec.transform(tst['text'])
print(X_5.shape, X_tst_5.shape)

(54879, 1024) (19617, 1024)


In [36]:
X_5[0, :50].todense()

matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.06917145,  0.        ,
          0.        ,  0.06917145,  0.        , -0.06917145, -0.06917145,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.06917145,
          0.        , -0.06917145,  0.        ,  0.        ,  0.06917145,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.06917145,  0.        ,  0.06917145,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

- keras의 text_to_word_sequence 사용, stopword 제거

In [37]:
vec = HashingVectorizer(tokenizer=text_to_word_sequence, ngram_range=(1, 3), n_features=2**10)
X_6 = vec.fit_transform(trn['text'])
X_tst_6 = vec.transform(tst['text'])
print(X_6.shape, X_tst_6.shape)

(54879, 1024) (19617, 1024)


In [38]:
X_6[0, :50].todense()

matrix([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.07881104,  0.        , -0.07881104,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.07881104,
          0.        , -0.07881104,  0.        ,  0.        ,  0.07881104,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.07881104,  0.        ,  0.07881104,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

#### 로지스틱회귀 모델 학습

In [39]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [40]:
y = trn.author.values
y.shape

(54879,)

In [41]:
p_val_ver1 = np.zeros((X_1.shape[0], n_class))
p_tst_ver1 = np.zeros((X_tst_1.shape[0], n_class))
p_val_ver2 = np.zeros((X_2.shape[0], n_class))
p_tst_ver2 = np.zeros((X_tst_2.shape[0], n_class))
p_val_ver3 = np.zeros((X_3.shape[0], n_class))
p_tst_ver3 = np.zeros((X_tst_3.shape[0], n_class))
p_val_ver4 = np.zeros((X_4.shape[0], n_class))
p_tst_ver4 = np.zeros((X_tst_4.shape[0], n_class))
p_val_ver5 = np.zeros((X_5.shape[0], n_class))
p_tst_ver5 = np.zeros((X_tst_5.shape[0], n_class))
p_val_ver6 = np.zeros((X_6.shape[0], n_class))
p_tst_ver6 = np.zeros((X_tst_6.shape[0], n_class))

for number, (X, test) in enumerate([(X_1, X_tst_1), (X_2, X_tst_2), (X_3, X_tst_3),
               (X_4, X_tst_4), (X_5, X_tst_5), (X_6, X_tst_6)],1):
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        X_train, X_val = X[i_trn], X[i_val]
        y_train, y_val = y[i_trn], y[i_val]
        
        lr_clf = LogisticRegression()
        lr_clf.fit(X_train,y_train)
       
        # Predict
        if number==1:
            p_val_ver1[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver1 += lr_clf.predict_proba(test) / n_class
        elif number==2:
            p_val_ver2[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver2 += lr_clf.predict_proba(test) / n_class
        elif number==3:
            p_val_ver3[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver3 += lr_clf.predict_proba(test) / n_class
        elif number==4:
            p_val_ver4[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver4 += lr_clf.predict_proba(test) / n_class
        elif number==5:
            p_val_ver5[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver5 += lr_clf.predict_proba(test) / n_class
        else:
            p_val_ver6[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver6 += lr_clf.predict_proba(test) / n_class
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver1 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver1, axis=1)) * 100:8.4f}%')
print(f'lr ver1 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver1):8.4f}')
print(f'lr ver2 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver2, axis=1)) * 100:8.4f}%')
print(f'lr ver2 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver2):8.4f}')
print(f'lr ver3 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver3, axis=1)) * 100:8.4f}%')
print(f'lr ver3 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver3):8.4f}')
print(f'lr ver4 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver4, axis=1)) * 100:8.4f}%')
print(f'lr ver4 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver4):8.4f}')
print(f'lr ver5 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver5, axis=1)) * 100:8.4f}%')
print(f'lr ver5 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver5):8.4f}')
print(f'lr ver6 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver6, axis=1)) * 100:8.4f}%')
print(f'lr ver6 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver6):8.4f}')

Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training

#### 제출 파일 생성 및 기타 파일 생성

In [42]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver1
sub[sub.columns] = p_tst_ver1
sub.to_csv(sub_ver1_file)

# Ver2
sub[sub.columns] = p_tst_ver2
sub.to_csv(sub_ver2_file)

# Ver3
sub[sub.columns] = p_tst_ver3
sub.to_csv(sub_ver3_file)

# Ver4
sub[sub.columns] = p_tst_ver4
sub.to_csv(sub_ver4_file)

# Ver5
sub[sub.columns] = p_tst_ver5
sub.to_csv(sub_ver5_file)

# Ver6
sub[sub.columns] = p_tst_ver6
sub.to_csv(sub_ver6_file)

In [43]:
# p_val 파일 생성 -> oof

# Ver1
np.savetxt(p_val_ver1_file, p_val_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_val_ver2_file, p_val_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_val_ver3_file, p_val_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_val_ver4_file, p_val_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_val_ver5_file, p_val_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_val_ver6_file, p_val_ver6, fmt='%.18f', delimiter=',')

In [44]:
# p_tst 파일 생성 -> test 

# Ver1
np.savetxt(p_tst_ver1_file, p_tst_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_tst_ver2_file, p_tst_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_tst_ver3_file, p_tst_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_tst_ver4_file, p_tst_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_tst_ver5_file, p_tst_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_tst_ver6_file, p_tst_ver6, fmt='%.18f', delimiter=',')

### 3-3. tf-idf

In [ ]:
algo_name = 'lr'
feature_name = 'tfidf'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'

p_val_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'

p_val_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
p_tst_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'

p_val_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
p_tst_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'

p_val_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'
p_tst_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'

p_val_ver5_file = val_dir / f'{model_name}_oof_pred_ver5.csv'
p_tst_ver5_file = tst_dir / f'{model_name}_test_pred_ver5.csv'

p_val_ver6_file = val_dir / f'{model_name}_oof_pred_ver6.csv'
p_tst_ver6_file = tst_dir / f'{model_name}_test_pred_ver6.csv'

sub_ver1_file = sub_dir / f'{model_name}_ver1.csv'
sub_ver2_file = sub_dir / f'{model_name}_ver2.csv'
sub_ver3_file = sub_dir / f'{model_name}_ver3.csv'
sub_ver4_file = sub_dir / f'{model_name}_ver4.csv'
sub_ver5_file = sub_dir / f'{model_name}_ver5.csv'
sub_ver6_file = sub_dir / f'{model_name}_ver6.csv'

#### TF-IDF 피쳐 생성

- nltk의 word_tokenize 사용

In [65]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_1 = vec.fit_transform(trn['text'])
X_tst_1 = vec.transform(tst['text'])
print(X_1.shape, X_tst_1.shape)

(54879, 5897) (19617, 5897)


In [66]:
X_1[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

- nltk의 WordPunctTokenizer 사용

In [67]:
vec = TfidfVectorizer(tokenizer=wordPunctTokenizer.tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_2 = vec.fit_transform(trn['text'])
X_tst_2 = vec.transform(tst['text'])
print(X_2.shape, X_tst_2.shape)

(54879, 5772) (19617, 5772)


In [68]:
X_2[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

- keras의 text_to_word_sequence 사용

In [69]:
vec = TfidfVectorizer(tokenizer=text_to_word_sequence, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_3 = vec.fit_transform(trn['text'])
X_tst_3 = vec.transform(tst['text'])
print(X_3.shape, X_tst_3.shape)

(54879, 3745) (19617, 3745)


In [70]:
X_3[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

- nltk의 word_tokenize 사용, stopword 제거

In [71]:
vec = TfidfVectorizer(tokenizer=word_tokenize, ngram_range=(1, 3), min_df=50)
X_4 = vec.fit_transform(trn['text'])
X_tst_4 = vec.transform(tst['text'])
print(X_4.shape, X_tst_4.shape)

(54879, 12254) (19617, 12254)


In [72]:
X_4[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

- nltk의 WordPunctTokenizer 사용, stopword 제거

In [73]:
vec = TfidfVectorizer(tokenizer=wordPunctTokenizer.tokenize, ngram_range=(1, 3), min_df=50)
X_5 = vec.fit_transform(trn['text'])
X_tst_5 = vec.transform(tst['text'])
print(X_5.shape, X_tst_5.shape)

(54879, 12267) (19617, 12267)


In [74]:
X_5[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

- keras의 text_to_word_sequence 사용, stopword 제거

In [75]:
vec = TfidfVectorizer(tokenizer=text_to_word_sequence, ngram_range=(1, 3), min_df=50)
X_6 = vec.fit_transform(trn['text'])
X_tst_6 = vec.transform(tst['text'])
print(X_6.shape, X_tst_6.shape)

(54879, 9555) (19617, 9555)


In [76]:
X_6[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

#### 로지스틱회귀 모델 학습

In [77]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [78]:
y = trn.author.values
y.shape

(54879,)

In [79]:
p_val_ver1 = np.zeros((X_1.shape[0], n_class))
p_tst_ver1 = np.zeros((X_tst_1.shape[0], n_class))
p_val_ver2 = np.zeros((X_2.shape[0], n_class))
p_tst_ver2 = np.zeros((X_tst_2.shape[0], n_class))
p_val_ver3 = np.zeros((X_3.shape[0], n_class))
p_tst_ver3 = np.zeros((X_tst_3.shape[0], n_class))
p_val_ver4 = np.zeros((X_4.shape[0], n_class))
p_tst_ver4 = np.zeros((X_tst_4.shape[0], n_class))
p_val_ver5 = np.zeros((X_5.shape[0], n_class))
p_tst_ver5 = np.zeros((X_tst_5.shape[0], n_class))
p_val_ver6 = np.zeros((X_6.shape[0], n_class))
p_tst_ver6 = np.zeros((X_tst_6.shape[0], n_class))

for X, test in [(X_1, X_tst_1), (X_2, X_tst_2), (X_3, X_tst_3),
               (X_4, X_tst_4), (X_5, X_tst_5), (X_6, X_tst_6)]:  
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        X_train, X_val = X[i_trn], X[i_val]
        y_train, y_val = y[i_trn], y[i_val]
        
        lr_clf = LogisticRegression()
        lr_clf.fit(X_train,y_train)
       
        # Predict
        if X.shape[1]==5897:
            p_val_ver1[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver1 += lr_clf.predict_proba(test) / n_class
        elif X.shape[1]==5772:
            p_val_ver2[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver2 += lr_clf.predict_proba(test) / n_class
        elif X.shape[1]==3745:
            p_val_ver3[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver3 += lr_clf.predict_proba(test) / n_class
        elif X.shape[1]==12254:
            p_val_ver4[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver4 += lr_clf.predict_proba(test) / n_class
        elif X.shape[1]==12267:
            p_val_ver5[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver5 += lr_clf.predict_proba(test) / n_class
        else:
            p_val_ver6[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver6 += lr_clf.predict_proba(test) / n_class
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver1 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver1, axis=1)) * 100:8.4f}%')
print(f'lr ver1 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver1):8.4f}')
print(f'lr ver2 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver2, axis=1)) * 100:8.4f}%')
print(f'lr ver2 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver2):8.4f}')
print(f'lr ver3 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver3, axis=1)) * 100:8.4f}%')
print(f'lr ver3 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver3):8.4f}')
print(f'lr ver4 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver4, axis=1)) * 100:8.4f}%')
print(f'lr ver4 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver4):8.4f}')
print(f'lr ver5 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver5, axis=1)) * 100:8.4f}%')
print(f'lr ver5 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver5):8.4f}')
print(f'lr ver6 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver6, axis=1)) * 100:8.4f}%')
print(f'lr ver6 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver6):8.4f}')

Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
Training

#### 제출 파일 생성 및 기타 파일 생성

In [80]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver1
sub[sub.columns] = p_tst_ver1
sub.to_csv(sub_ver1_file)

# Ver2
sub[sub.columns] = p_tst_ver2
sub.to_csv(sub_ver2_file)

# Ver3
sub[sub.columns] = p_tst_ver3
sub.to_csv(sub_ver3_file)

# Ver4
sub[sub.columns] = p_tst_ver4
sub.to_csv(sub_ver4_file)

# Ver5
sub[sub.columns] = p_tst_ver5
sub.to_csv(sub_ver5_file)

# Ver6
sub[sub.columns] = p_tst_ver6
sub.to_csv(sub_ver6_file)

In [81]:
# p_val 파일 생성 -> oof

# Ver1
np.savetxt(p_val_ver1_file, p_val_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_val_ver2_file, p_val_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_val_ver3_file, p_val_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_val_ver4_file, p_val_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_val_ver5_file, p_val_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_val_ver6_file, p_val_ver6, fmt='%.18f', delimiter=',')

In [82]:
# p_tst 파일 생성 -> test 

# Ver1
np.savetxt(p_tst_ver1_file, p_tst_ver1, fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(p_tst_ver2_file, p_tst_ver2, fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(p_tst_ver3_file, p_tst_ver3, fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(p_tst_ver4_file, p_tst_ver4, fmt='%.18f', delimiter=',')

# Ver5
np.savetxt(p_tst_ver5_file, p_tst_ver5, fmt='%.18f', delimiter=',')

# Ver6
np.savetxt(p_tst_ver6_file, p_tst_ver6, fmt='%.18f', delimiter=',')

### 3-4. tf-idf-pca

In [ ]:
algo_name = 'lr'
feature_name = 'tfidf-pca' #학습할 피처- tfidf 피처를 pca한 데이터
feature_target = 'feature_target'

model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv' #학습할 피처 가져오기
feature_target_file = feature_dir / f'{feature_target}.csv' #target 데이터 가져오기

p_val_ver7_file = val_dir / f'{model_name}_oof_pred_ver7.csv'
p_tst_ver7_file = tst_dir / f'{model_name}_test_pred_ver7.csv'
sub_ver7_file = sub_dir / f'{model_name}_ver7.csv'

In [ ]:
X = pd.read_csv(feature_file, index_col=0).values
y = pd.read_csv(feature_target_file,index_col=0).values

X_7 = X[:54879]
X_tst_7 = X[54879:]

In [ ]:
## 로지스틱회귀 모델 학습

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

p_val_ver7 = np.zeros((X_7.shape[0], n_class))
p_tst_ver7 = np.zeros((X_tst_7.shape[0], n_class))


for X, test in [(X_7, X_tst_7)]:  
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        X_train, X_val = X[i_trn], X[i_val]
        y_train, y_val = y[i_trn], y[i_val]
        
        lr_clf = LogisticRegression()
        lr_clf.fit(X_train,y_train)
       
        # Predict
        if X.shape[1]==500:
            p_val_ver7[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver7 += lr_clf.predict_proba(test) / n_class
        
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver7 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver7, axis=1)) * 100:8.4f}%')
print(f'lr ver7 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver7):8.4f}')


## 제출 파일 생성 및 기타 파일 생성

# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver7
sub[sub.columns] = p_tst_ver7
sub.to_csv(sub_ver7_file)

# p_val 파일 생성 -> oof

# Ver7
np.savetxt(p_val_ver7_file, p_val_ver7, fmt='%.18f', delimiter=',')

# p_tst 파일 생성 -> test 
# Ver7
np.savetxt(p_tst_ver7_file, p_tst_ver7, fmt='%.18f', delimiter=',')


## 4. Transformer

### 4-1. Ver1

In [ ]:
algorithm_name = 'transformer'
feature_name = 'feature'
model_name = f'{algorithm_name}_{feature_name}'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_Ver3_file = feature_dir / f'{feature_name}_Ver3.csv'
feature_Ver4_file = feature_dir / f'{feature_name}_Ver4.csv'


transformer_oof_pred_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
transformer_oof_pred_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
transformer_oof_pred_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
transformer_oof_pred_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'


transformer_test_pred_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'
transformer_test_pred_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'
transformer_test_pred_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'
transformer_test_pred_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'


transformer_submission_ver1_file = sub_dir / f'{model_name}_submission_Ver1.csv'
transformer_submission_ver2_file = sub_dir / f'{model_name}_submission_Ver2.csv'
transformer_submission_ver3_file = sub_dir / f'{model_name}_submission_Ver3.csv'
transformer_submission_ver4_file = sub_dir / f'{model_name}_submission_Ver4.csv'

#### Ver 1 데이터 로드

In [7]:
dataset = pd.read_csv(feature_Ver1_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he was almost choking there was so much so muc...,3.0
1,your sister asked for it i suppose,2.0
2,she was engaged one day as she walked in peru...,1.0
3,the captain was in the porch keeping himself c...,4.0
4,have mercy gentlemen odin flung up his hands d...,3.0


In [8]:
# train set
Ver1_X = dataset.loc[dataset[target_col] != -1 , :]
Ver1_X.drop(columns=target_col,inplace=True,axis=1)
Ver1_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver1_y.astype(int)

# test set
Ver1_test = dataset.loc[dataset[target_col] == -1, :]
Ver1_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver1_X.shape, Ver1_y.shape, Ver1_test.shape)

(54879, 1) (54879,) (19617, 1)


#### Ver 2 데이터 로드

In [9]:
dataset = pd.read_csv(feature_Ver2_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged one day walked perusing janes last let...,1.0
3,captain porch keeping carefully way treacherou...,4.0
4,mercy gentlemen odin flung hands dont write an...,3.0


In [10]:
# train set
Ver2_X = dataset.loc[dataset[target_col] != -1 , :]
Ver2_X.drop(columns=target_col,inplace=True,axis=1)
Ver2_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver2_y.astype(int)

# test set
Ver2_test = dataset.loc[dataset[target_col] == -1, :]
Ver2_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver2_X.shape, Ver2_y.shape, Ver2_test.shape)

(54879, 1) (54879,) (19617, 1)


#### Ver3 데이터 로드

In [11]:
dataset = pd.read_csv(feature_Ver3_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged day walked perusing janes last letter ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,mercy gentlemen flung up hands dont write anyw...,3.0


In [12]:
# train set
Ver3_X = dataset.loc[dataset[target_col] != -1 , :]
Ver3_X.drop(columns=target_col,inplace=True,axis=1)
Ver3_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver3_y.astype(int)

# test set
Ver3_test = dataset.loc[dataset[target_col] == -1, :]
Ver3_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver3_X.shape, Ver3_y.shape, Ver3_test.shape)

(54879, 1) (54879,) (19617, 1)


#### Ver4 데이터 로드

In [13]:
dataset = pd.read_csv(feature_Ver4_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he almost choking there so much so much he wan...,3.0
1,sister asked for it suppose,2.0
2,she engaged one day she walked perusing janes ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,have mercy gentlemen odin flung up hands dont ...,3.0


In [14]:
# train set
Ver4_X = dataset.loc[dataset[target_col] != -1 , :]
Ver4_X.drop(columns=target_col,inplace=True,axis=1)
Ver4_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver4_y.astype(int)

# test set
Ver4_test = dataset.loc[dataset[target_col] == -1, :]
Ver4_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver4_X.shape, Ver4_y.shape, Ver4_test.shape)

(54879, 1) (54879,) (19617, 1)


#### 모델 학습

In [15]:
vocab_size = 30000
maxlen = 500
embed_dim = 128
num_heads = 8  # Number of attention heads
padding_type='post'

In [16]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output
    
    
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
    
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [17]:
def get_model():
    ff_dim = 64  # Hidden layer size in feed forward network inside transformer

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(n_class, activation="softmax")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.001))
    return model

In [18]:
datasets = [ (Ver1_X, Ver1_test, Ver1_y), (Ver2_X, Ver2_test, Ver2_y),
            (Ver3_X, Ver3_test, Ver3_y), (Ver4_X, Ver4_test, Ver3_y)]

transformer_oof_preds = []
transformer_test_preds = []

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

for number ,(X, test, y) in enumerate(datasets,1):
    print(f'start : {number}')
    # 토큰화
    X_train = np.array([x for x in X['text']])
    X_test = np.array([x for x in test['text']])
    y = np.array(y.values)
    
    tokenizer = Tokenizer(num_words = vocab_size)
    tokenizer.fit_on_texts(X_train)
    word_index = tokenizer.word_index
    
    # 시퀸스화 + 패딩
    train_sequences = tokenizer.texts_to_sequences(X_train)
    test_sequences = tokenizer.texts_to_sequences(X_test)
    trn = pad_sequences(train_sequences, padding=padding_type, maxlen=maxlen)
    tst = pad_sequences(test_sequences, padding=padding_type, maxlen=maxlen)
    print(trn.shape, tst.shape)
    
    # oof , test 저장
    transformer_oof_pred = np.zeros((trn.shape[0], n_class))
    transformer_test_pred = np.zeros((tst.shape[0], n_class))
    
    for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
        print(f'traing model for CV #{i}')
        clf = get_model()
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=200,
            batch_size=32,
            callbacks=[es])
        
        transformer_oof_pred[i_val, :] = clf.predict(trn[i_val])
        transformer_test_pred += clf.predict(tst) / n_fold
        
        del clf
        clear_session()
        gc.collect()
    
    transformer_oof_preds.append(transformer_oof_pred)
    transformer_test_preds.append(transformer_test_pred)
        
    print(f'end : {number}')

start : 1
(54879, 500) (19617, 500)
traing model for CV #1
Epoch 1/200
1372/1372 [==============================] - 124s 91ms/step - loss: 1.3079 - val_loss: 0.9779
Epoch 2/200
1372/1372 [==============================] - 123s 90ms/step - loss: 0.8065 - val_loss: 0.7319
Epoch 3/200
1372/1372 [==============================] - 124s 90ms/step - loss: 0.5109 - val_loss: 0.7323
Epoch 4/200
1372/1372 [==============================] - 126s 92ms/step - loss: 0.3523 - val_loss: 0.8557
Epoch 5/200
1372/1372 [==============================] - 126s 92ms/step - loss: 0.2729 - val_loss: 1.0067
Epoch 00005: early stopping
traing model for CV #2
Epoch 1/200
1372/1372 [==============================] - 127s 92ms/step - loss: 1.3197 - val_loss: 1.0526
Epoch 2/200
1372/1372 [==============================] - 127s 92ms/step - loss: 0.8819 - val_loss: 0.9184
Epoch 3/200
1372/1372 [==============================] - 127s 92ms/step - loss: 0.5883 - val_loss: 0.7622
Epoch 4/200
1372/1372 [===================

1372/1372 [==============================] - 125s 91ms/step - loss: 0.8458 - val_loss: 0.7780
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.5156 - val_loss: 0.7713
Epoch 4/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.3418 - val_loss: 0.9017
Epoch 5/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.2630 - val_loss: 1.0048
Epoch 6/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.2112 - val_loss: 1.2188
Epoch 00006: early stopping
traing model for CV #5
Epoch 1/200
1372/1372 [==============================] - 124s 91ms/step - loss: 1.2909 - val_loss: 1.0352
Epoch 2/200
1372/1372 [==============================] - 123s 90ms/step - loss: 0.8145 - val_loss: 0.7957
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.5062 - val_loss: 0.7875
Epoch 4/200
1372/1372 [==============================] - 124s 90ms/step - loss: 0.3520 - val_loss: 0.9214
Epoch 5

Epoch 00006: early stopping
traing model for CV #3
Epoch 1/200
1372/1372 [==============================] - 126s 92ms/step - loss: 1.2394 - val_loss: 0.9326
Epoch 2/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.6977 - val_loss: 0.7156
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.4304 - val_loss: 0.7853
Epoch 4/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.3051 - val_loss: 0.9144
Epoch 5/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.2425 - val_loss: 1.0594
Epoch 00005: early stopping
traing model for CV #4
Epoch 1/200
1372/1372 [==============================] - 126s 92ms/step - loss: 1.3167 - val_loss: 1.1634
Epoch 2/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.7873 - val_loss: 0.7476
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.4831 - val_loss: 0.7703
Epoch 4/200
1372/1372 [===========================

In [19]:
for i,j in enumerate(transformer_oof_preds,1):
    print(f'ver{i} logloss = {log_loss(pd.get_dummies(y),j):8.4f}')
    print(f'ver{i} accruacy = {accuracy_score(y, np.argmax(j,axis=1))*100:8.4f}')

ver1 logloss =   0.7455
ver1 logloss =  72.5195
ver2 logloss =   0.7689
ver2 logloss =  72.2426
ver3 logloss =   0.7867
ver3 logloss =  71.1948
ver4 logloss =   0.7480
ver4 logloss =  72.9113


#### 제출 파일 및 기타 파일 생성

In [21]:
# submission 파일 생성

sub = pd.read_csv(sample_file,index_col=0)

# Ver1 
sub[sub.columns] = transformer_test_preds[0]
sub.to_csv(transformer_submission_ver1_file)

# Ver2
sub[sub.columns] = transformer_test_preds[1]
sub.to_csv(transformer_submission_ver2_file)

# Ver3
sub[sub.columns] = transformer_test_preds[2]
sub.to_csv(transformer_submission_ver3_file)
           
# Ver4
sub[sub.columns] = transformer_test_preds[3]
sub.to_csv(transformer_submission_ver4_file)

In [22]:
# transformer_oof_pred 파일 생성

# Ver1
np.savetxt(transformer_oof_pred_ver1_file, transformer_oof_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(transformer_oof_pred_ver2_file, transformer_oof_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(transformer_oof_pred_ver3_file, transformer_oof_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(transformer_oof_pred_ver4_file, transformer_oof_preds[3],fmt='%.18f', delimiter=',')

In [23]:
# transformer_test_pred 파일 생성

# Ver1
np.savetxt(transformer_test_pred_ver1_file, transformer_test_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(transformer_test_pred_ver2_file, transformer_test_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(transformer_test_pred_ver3_file, transformer_test_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(transformer_test_pred_ver4_file, transformer_test_preds[3],fmt='%.18f', delimiter=',')

### 4-2. Ver2

In [ ]:
algorithm_name = 'transformer'
feature_name = 'feature2'
model_name = f'{algorithm_name}_{feature_name}'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_Ver3_file = feature_dir / f'{feature_name}_Ver3.csv'
feature_Ver4_file = feature_dir / f'{feature_name}_Ver4.csv'


transformer_oof_pred_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
transformer_oof_pred_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
transformer_oof_pred_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
transformer_oof_pred_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'


transformer_test_pred_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'
transformer_test_pred_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'
transformer_test_pred_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'
transformer_test_pred_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'


transformer_submission_ver1_file = sub_dir / f'{model_name}_submission_Ver1.csv'
transformer_submission_ver2_file = sub_dir / f'{model_name}_submission_Ver2.csv'
transformer_submission_ver3_file = sub_dir / f'{model_name}_submission_Ver3.csv'
transformer_submission_ver4_file = sub_dir / f'{model_name}_submission_Ver4.csv'

#### Ver 1 데이터 로드

In [7]:
dataset = pd.read_csv(feature_Ver1_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he was almost choking there was so much so muc...,3.0
1,your sister asked for it i suppose,2.0
2,she was engaged one day as she walked in peru...,1.0
3,the captain was in the porch keeping himself c...,4.0
4,have mercy gentlemen odin flung up his hands d...,3.0


In [8]:
# train set
Ver1_X = dataset.loc[dataset[target_col] != -1 , :]
Ver1_X.drop(columns=target_col,inplace=True,axis=1)
Ver1_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver1_y.astype(int)

# test set
Ver1_test = dataset.loc[dataset[target_col] == -1, :]
Ver1_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver1_X.shape, Ver1_y.shape, Ver1_test.shape)

(54879, 1) (54879,) (19617, 1)


#### Ver 2 데이터 로드

In [9]:
dataset = pd.read_csv(feature_Ver2_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged one day walked perusing janes last let...,1.0
3,captain porch keeping carefully way treacherou...,4.0
4,mercy gentlemen odin flung hands dont write an...,3.0


In [10]:
# train set
Ver2_X = dataset.loc[dataset[target_col] != -1 , :]
Ver2_X.drop(columns=target_col,inplace=True,axis=1)
Ver2_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver2_y.astype(int)

# test set
Ver2_test = dataset.loc[dataset[target_col] == -1, :]
Ver2_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver2_X.shape, Ver2_y.shape, Ver2_test.shape)

(54879, 1) (54879,) (19617, 1)


#### Ver3 데이터 로드

In [11]:
dataset = pd.read_csv(feature_Ver3_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged day walked perusing janes last letter ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,mercy gentlemen flung up hands dont write anyw...,3.0


In [12]:
# train set
Ver3_X = dataset.loc[dataset[target_col] != -1 , :]
Ver3_X.drop(columns=target_col,inplace=True,axis=1)
Ver3_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver3_y.astype(int)

# test set
Ver3_test = dataset.loc[dataset[target_col] == -1, :]
Ver3_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver3_X.shape, Ver3_y.shape, Ver3_test.shape)

(54879, 1) (54879,) (19617, 1)


#### Ver4 데이터 로드

In [13]:
dataset = pd.read_csv(feature_Ver4_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he almost choking there so much so much he wan...,3.0
1,sister asked for it suppose,2.0
2,she engaged one day she walked perusing janes ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,have mercy gentlemen odin flung up hands dont ...,3.0


In [14]:
# train set
Ver4_X = dataset.loc[dataset[target_col] != -1 , :]
Ver4_X.drop(columns=target_col,inplace=True,axis=1)
Ver4_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver4_y.astype(int)

# test set
Ver4_test = dataset.loc[dataset[target_col] == -1, :]
Ver4_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver4_X.shape, Ver4_y.shape, Ver4_test.shape)

(54879, 1) (54879,) (19617, 1)


#### 모델 학습

In [15]:
vocab_size = 30000
maxlen = 500
embed_dim = 128
num_heads = 8  # Number of attention heads
padding_type='post'

In [16]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output
    
    
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
    
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [17]:
def get_model():
    ff_dim = 32  # Hidden layer size in feed forward network inside transformer

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation="relu")(x)
    outputs = layers.Dense(n_class, activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.001))
    return model

In [18]:
datasets = [ (Ver1_X, Ver1_test, Ver1_y), (Ver2_X, Ver2_test, Ver2_y),
            (Ver3_X, Ver3_test, Ver3_y), (Ver4_X, Ver4_test, Ver3_y)]

transformer_oof_preds = []
transformer_test_preds = []

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

for number ,(X, test, y) in enumerate(datasets,1):
    print(f'start : {number}')
    # 토큰화
    X_train = np.array([x for x in X['text']])
    X_test = np.array([x for x in test['text']])
    y = np.array(y.values)
    
    tokenizer = Tokenizer(num_words = vocab_size)
    tokenizer.fit_on_texts(X_train)
    word_index = tokenizer.word_index
    
    # 시퀸스화 + 패딩
    train_sequences = tokenizer.texts_to_sequences(X_train)
    test_sequences = tokenizer.texts_to_sequences(X_test)
    trn = pad_sequences(train_sequences, padding=padding_type, maxlen=maxlen)
    tst = pad_sequences(test_sequences, padding=padding_type, maxlen=maxlen)
    print(trn.shape, tst.shape)
    
    # oof , test 저장
    transformer_oof_pred = np.zeros((trn.shape[0], n_class))
    transformer_test_pred = np.zeros((tst.shape[0], n_class))
    
    for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
        print(f'traing model for CV #{i}')
        clf = get_model()
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=200,
            batch_size=32,
            callbacks=[es])
        
        transformer_oof_pred[i_val, :] = clf.predict(trn[i_val])
        transformer_test_pred += clf.predict(tst) / n_fold
        
        del clf
        clear_session()
        gc.collect()
    
    transformer_oof_preds.append(transformer_oof_pred)
    transformer_test_preds.append(transformer_test_pred)
        
    print(f'end : {number}')

start : 1
(54879, 500) (19617, 500)
traing model for CV #1
Epoch 1/200
1372/1372 [==============================] - 124s 91ms/step - loss: 1.3079 - val_loss: 0.9779
Epoch 2/200
1372/1372 [==============================] - 123s 90ms/step - loss: 0.8065 - val_loss: 0.7319
Epoch 3/200
1372/1372 [==============================] - 124s 90ms/step - loss: 0.5109 - val_loss: 0.7323
Epoch 4/200
1372/1372 [==============================] - 126s 92ms/step - loss: 0.3523 - val_loss: 0.8557
Epoch 5/200
1372/1372 [==============================] - 126s 92ms/step - loss: 0.2729 - val_loss: 1.0067
Epoch 00005: early stopping
traing model for CV #2
Epoch 1/200
1372/1372 [==============================] - 127s 92ms/step - loss: 1.3197 - val_loss: 1.0526
Epoch 2/200
1372/1372 [==============================] - 127s 92ms/step - loss: 0.8819 - val_loss: 0.9184
Epoch 3/200
1372/1372 [==============================] - 127s 92ms/step - loss: 0.5883 - val_loss: 0.7622
Epoch 4/200
1372/1372 [===================

1372/1372 [==============================] - 125s 91ms/step - loss: 0.8458 - val_loss: 0.7780
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.5156 - val_loss: 0.7713
Epoch 4/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.3418 - val_loss: 0.9017
Epoch 5/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.2630 - val_loss: 1.0048
Epoch 6/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.2112 - val_loss: 1.2188
Epoch 00006: early stopping
traing model for CV #5
Epoch 1/200
1372/1372 [==============================] - 124s 91ms/step - loss: 1.2909 - val_loss: 1.0352
Epoch 2/200
1372/1372 [==============================] - 123s 90ms/step - loss: 0.8145 - val_loss: 0.7957
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.5062 - val_loss: 0.7875
Epoch 4/200
1372/1372 [==============================] - 124s 90ms/step - loss: 0.3520 - val_loss: 0.9214
Epoch 5

Epoch 00006: early stopping
traing model for CV #3
Epoch 1/200
1372/1372 [==============================] - 126s 92ms/step - loss: 1.2394 - val_loss: 0.9326
Epoch 2/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.6977 - val_loss: 0.7156
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.4304 - val_loss: 0.7853
Epoch 4/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.3051 - val_loss: 0.9144
Epoch 5/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.2425 - val_loss: 1.0594
Epoch 00005: early stopping
traing model for CV #4
Epoch 1/200
1372/1372 [==============================] - 126s 92ms/step - loss: 1.3167 - val_loss: 1.1634
Epoch 2/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.7873 - val_loss: 0.7476
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.4831 - val_loss: 0.7703
Epoch 4/200
1372/1372 [===========================

In [19]:
for i,j in enumerate(transformer_oof_preds,1):
    print(f'ver{i} logloss = {log_loss(pd.get_dummies(y),j):8.4f}')
    print(f'ver{i} accruacy = {accuracy_score(y, np.argmax(j,axis=1))*100:8.4f}')

ver1 logloss =   0.7455
ver1 logloss =  72.5195
ver2 logloss =   0.7689
ver2 logloss =  72.2426
ver3 logloss =   0.7867
ver3 logloss =  71.1948
ver4 logloss =   0.7480
ver4 logloss =  72.9113


#### 제출 파일 및 기타 파일 생성

In [21]:
# submission 파일 생성

sub = pd.read_csv(sample_file,index_col=0)

# Ver1 
sub[sub.columns] = transformer_test_preds[0]
sub.to_csv(transformer_submission_ver1_file)

# Ver2
sub[sub.columns] = transformer_test_preds[1]
sub.to_csv(transformer_submission_ver2_file)

# Ver3
sub[sub.columns] = transformer_test_preds[2]
sub.to_csv(transformer_submission_ver3_file)
           
# Ver4
sub[sub.columns] = transformer_test_preds[3]
sub.to_csv(transformer_submission_ver4_file)

In [22]:
# transformer_oof_pred 파일 생성

# Ver1
np.savetxt(transformer_oof_pred_ver1_file, transformer_oof_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(transformer_oof_pred_ver2_file, transformer_oof_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(transformer_oof_pred_ver3_file, transformer_oof_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(transformer_oof_pred_ver4_file, transformer_oof_preds[3],fmt='%.18f', delimiter=',')

In [23]:
# transformer_test_pred 파일 생성

# Ver1
np.savetxt(transformer_test_pred_ver1_file, transformer_test_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(transformer_test_pred_ver2_file, transformer_test_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(transformer_test_pred_ver3_file, transformer_test_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(transformer_test_pred_ver4_file, transformer_test_preds[3],fmt='%.18f', delimiter=',')

## 5. LSTM

In [ ]:
algorithm_name = 'lstm'
feature_name = 'feature'
model_name = f'{algorithm_name}_{feature_name}'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_Ver3_file = feature_dir / f'{feature_name}_Ver3.csv'
feature_Ver4_file = feature_dir / f'{feature_name}_Ver4.csv'


lstm_oof_pred_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
lstm_oof_pred_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
lstm_oof_pred_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
lstm_oof_pred_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'


lstm_test_pred_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'
lstm_test_pred_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'
lstm_test_pred_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'
lstm_test_pred_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'


lstm_submission_ver1_file = sub_dir / f'{model_name}_submission_Ver1.csv'
lstm_submission_ver2_file = sub_dir / f'{model_name}_submission_Ver2.csv'
lstm_submission_ver3_file = sub_dir / f'{model_name}_submission_Ver3.csv'
lstm_submission_ver4_file = sub_dir / f'{model_name}_submission_Ver4.csv'

### Ver 1 데이터 로드

In [7]:
dataset = pd.read_csv(feature_Ver1_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he was almost choking there was so much so muc...,3.0
1,your sister asked for it i suppose,2.0
2,she was engaged one day as she walked in peru...,1.0
3,the captain was in the porch keeping himself c...,4.0
4,have mercy gentlemen odin flung up his hands d...,3.0


In [8]:
# train set
Ver1_X = dataset.loc[dataset[target_col] != -1 , :]
Ver1_X.drop(columns=target_col,inplace=True,axis=1)
Ver1_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver1_y.astype(int)

# test set
Ver1_test = dataset.loc[dataset[target_col] == -1, :]
Ver1_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver1_X.shape, Ver1_y.shape, Ver1_test.shape)

(54879, 1) (54879,) (19617, 1)


### Ver 2 데이터 로드

In [9]:
dataset = pd.read_csv(feature_Ver2_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged one day walked perusing janes last let...,1.0
3,captain porch keeping carefully way treacherou...,4.0
4,mercy gentlemen odin flung hands dont write an...,3.0


In [10]:
# train set
Ver2_X = dataset.loc[dataset[target_col] != -1 , :]
Ver2_X.drop(columns=target_col,inplace=True,axis=1)
Ver2_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver2_y.astype(int)

# test set
Ver2_test = dataset.loc[dataset[target_col] == -1, :]
Ver2_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver2_X.shape, Ver2_y.shape, Ver2_test.shape)

(54879, 1) (54879,) (19617, 1)


### Ver3 데이터 로드

In [11]:
dataset = pd.read_csv(feature_Ver3_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged day walked perusing janes last letter ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,mercy gentlemen flung up hands dont write anyw...,3.0


In [12]:
# train set
Ver3_X = dataset.loc[dataset[target_col] != -1 , :]
Ver3_X.drop(columns=target_col,inplace=True,axis=1)
Ver3_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver3_y.astype(int)

# test set
Ver3_test = dataset.loc[dataset[target_col] == -1, :]
Ver3_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver3_X.shape, Ver3_y.shape, Ver3_test.shape)

(54879, 1) (54879,) (19617, 1)


### Ver4 데이터 로드

In [13]:
dataset = pd.read_csv(feature_Ver4_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he almost choking there so much so much he wan...,3.0
1,sister asked for it suppose,2.0
2,she engaged one day she walked perusing janes ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,have mercy gentlemen odin flung up hands dont ...,3.0


In [14]:
# train set
Ver4_X = dataset.loc[dataset[target_col] != -1 , :]
Ver4_X.drop(columns=target_col,inplace=True,axis=1)
Ver4_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver4_y.astype(int)

# test set
Ver4_test = dataset.loc[dataset[target_col] == -1, :]
Ver4_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver4_X.shape, Ver4_y.shape, Ver4_test.shape)

(54879, 1) (54879,) (19617, 1)


### 모델 학습

In [15]:
#파라미터 설정
vocab_size = 30000
embedding_dim = 128
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [16]:
def get_model():
    #가벼운 NLP모델 생성
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(5, activation='softmax')
    ])
    
    # compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=.01))
    return model

In [17]:
datasets = [ (Ver1_X, Ver1_test, Ver1_y), (Ver2_X, Ver2_test, Ver2_y),
            (Ver3_X, Ver3_test, Ver3_y), (Ver4_X, Ver4_test, Ver3_y)]

lstm_oof_preds = []
lstm_test_preds = []

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

for number ,(X, test, y) in enumerate(datasets,1):
    print(f'start : {number}')
    # 토큰화
    X_train = np.array([x for x in X['text']])
    X_test = np.array([x for x in test['text']])
    y = np.array(y.values)
    
    tokenizer = Tokenizer(num_words = vocab_size)
    tokenizer.fit_on_texts(X_train)
    word_index = tokenizer.word_index
    
    # 시퀸스화 + 패딩
    train_sequences = tokenizer.texts_to_sequences(X_train)
    test_sequences = tokenizer.texts_to_sequences(X_test)
    trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
    tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
    print(trn.shape, tst.shape)
    
    # oof , test 저장
    lstm_oof_pred = np.zeros((trn.shape[0], n_class))
    lstm_test_pred = np.zeros((tst.shape[0], n_class))
    
    for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
        print(f'traing model for CV #{i}')
        clf = get_model()
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=200,
            batch_size=1024,
            callbacks=[es])
        
        lstm_oof_pred[i_val, :] = clf.predict(trn[i_val])
        lstm_test_pred += clf.predict(tst) / n_fold
        
        del clf
        clear_session()
        gc.collect()
    
    lstm_oof_preds.append(lstm_oof_pred)
    lstm_test_preds.append(lstm_test_pred)
        
    print(f'end : {number}')

start : 1
(54879, 500) (19617, 500)
traing model for CV #1
Epoch 1/200
43/43 [==============================] - 23s 537ms/step - loss: 1.3609 - val_loss: 1.0124
Epoch 2/200
43/43 [==============================] - 22s 507ms/step - loss: 0.7708 - val_loss: 0.7674
Epoch 3/200
43/43 [==============================] - 22s 508ms/step - loss: 0.4995 - val_loss: 0.7320
Epoch 4/200
43/43 [==============================] - 22s 508ms/step - loss: 0.3544 - val_loss: 0.7609
Epoch 5/200
43/43 [==============================] - 22s 511ms/step - loss: 0.2760 - val_loss: 0.8571
Epoch 6/200
43/43 [==============================] - 22s 510ms/step - loss: 0.2265 - val_loss: 0.9220
Epoch 00006: early stopping
traing model for CV #2
Epoch 1/200
43/43 [==============================] - 23s 536ms/step - loss: 1.2759 - val_loss: 1.0086
Epoch 2/200
43/43 [==============================] - 22s 510ms/step - loss: 0.7133 - val_loss: 0.7749
Epoch 3/200
43/43 [==============================] - 22s 509ms/step - loss

In [18]:
for i,j in enumerate(lstm_oof_preds,1):
    print(f'ver{i} logloss = {log_loss(pd.get_dummies(y),j):8.4f}')
    print(f'ver{i} accuracy = {accuracy_score(y, np.argmax(j,axis=1))*100:8.4f}')

ver1 logloss =   0.7444
ver1 logloss =  73.6712
ver2 logloss =   0.7431
ver2 logloss =  72.7765
ver3 logloss =   0.7439
ver3 logloss =  72.5050
ver4 logloss =   0.7379
ver4 logloss =  73.3213


### 제출 파일 및 기타 파일 생성

In [19]:
# submission 파일 생성

sub = pd.read_csv(sample_file,index_col=0)

# Ver1 
sub[sub.columns] = lstm_test_preds[0]
sub.to_csv(lstm_submission_ver1_file)

# Ver2
sub[sub.columns] = lstm_test_preds[1]
sub.to_csv(lstm_submission_ver2_file)

# Ver3
sub[sub.columns] = lstm_test_preds[2]
sub.to_csv(lstm_submission_ver3_file)
           
# Ver4
sub[sub.columns] = lstm_test_preds[3]
sub.to_csv(lstm_submission_ver4_file)

In [20]:
# lstm_oof_pred 파일 생성

# Ver1
np.savetxt(lstm_oof_pred_ver1_file, lstm_oof_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(lstm_oof_pred_ver2_file, lstm_oof_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(lstm_oof_pred_ver3_file, lstm_oof_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(lstm_oof_pred_ver4_file, lstm_oof_preds[3],fmt='%.18f', delimiter=',')

In [21]:
# lstm_test_pred 파일 생성

# Ver1
np.savetxt(lstm_test_pred_ver1_file, lstm_test_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(lstm_test_pred_ver2_file, lstm_test_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(lstm_test_pred_ver3_file, lstm_test_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(lstm_test_pred_ver4_file, lstm_test_preds[3],fmt='%.18f', delimiter=',')

## 6. LGBM

### 6-1. tf-idf

In [45]:
algo_name = 'lgbm'
feature_name = 'tfidf'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_file =  tst_dir / f'{model_name}_test_pred_ver1.csv'
sub_file = sub_dir / f'{model_name}_submission_ver1.csv'

#### Bag-of-Words 피처 생성

In [46]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_cnt = vec.fit_transform(trn['text'])
print(X_cnt.shape)

(54879, 2685)


In [47]:
X_cnt[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

In [48]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X = vec.fit_transform(trn['text'])
X_tst = vec.transform(tst['text'])
print(X.shape, X_tst.shape)

(54879, 5897) (19617, 5897)


In [49]:
X[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

#### LGBM 모델 학습

In [18]:
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [ ]:
p = np.zeros((X.shape[0], n_class))
p_tst = np.zeros((X_tst.shape[0], n_class))

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X, y)) :
    
    print('## cv : ', n_fold)
    
    # split
    trn_x, trn_y = X[trn_idx], y[trn_idx]
    val_x, val_y = X[val_idx], y[val_idx]
    
    print('모델 생성')
    lr_clf = LGBMClassifier(learning_rate = 0.005, max_depth  = 40, n_estimators  = 15000,
                            num_leaves = 512,
                           feature_fraction = 0.9,
                           bagging_fraction = 0.7,      
                           seed = seed)  
    
    print('모델 생성 후 학습 시작')
    lr_clf.fit(
            trn_x, trn_y,
            eval_set = [(val_x, val_y)],
            eval_metric = 'logloss',
            verbose =100, 
            early_stopping_rounds = 30
        )
    
    p_tst += lr_clf.predict_proba(X_tst) / folds.n_splits
    p[val_idx, :] = lr_clf.predict_proba(val_x)
    

In [ ]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

In [ ]:
np.savetxt(p_val_file, p, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

#### 제출 파일 생성

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

In [ ]:
sub[sub.columns] = p_tst
sub.head()

In [ ]:
sub.to_csv(sub_file)

### 6.2. tf-idf-pca

In [ ]:
algo_name = 'lgbm'
feature_name = 'tfidf-pca'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
p_tst_file =  tst_dir / f'{model_name}_test_pred_ver1.csv'
sub_file = sub_dir / f'{model_name}_submission_ver1.csv'

#### Bag-of-Words 피처 생성

In [ ]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_cnt = vec.fit_transform(trn['text'])
print(X_cnt.shape)

In [ ]:
X_cnt[0, :50].todense()

In [ ]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X = vec.fit_transform(trn['text'])
X_tst = vec.transform(tst['text'])
print(X.shape, X_tst.shape)

In [ ]:
X[0, :50].todense()

#### PCA

In [ ]:
X.shape

In [ ]:
n_batches = 39
inc_pca = IncrementalPCA(n_components=500)#temp.size*0.5)
for batch_x in np.array_split(X.todense(), n_batches):
    print(".", end="") # not shown in the book
    inc_pca.partial_fit(batch_x)

# incrementalPCA로 PCA 값을 생성
X_inc = inc_pca.transform(X.todense())    

inc_pca = IncrementalPCA(n_components=500)#temp.size*0.5)
for batch_x in np.array_split(X_tst.todense(), n_batches):
    print(".", end="") # not shown in the book
    inc_pca.partial_fit(batch_x)
    
# incrementalPCA로 PCA 값을 생성
X_tst_inc = inc_pca.transform(X_tst.todense())

In [ ]:
X_inc.shape, X_tst_inc.shape

#### LGBM 모델 학습

In [ ]:
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [ ]:
y = trn.author.values
y.shape

In [ ]:
p = np.zeros((X.shape[0], n_class))
p_tst = np.zeros((X_tst.shape[0], n_class))

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X_inc, y)) :
    
    print('## cv : ', n_fold)
    
    # split
    trn_x, trn_y = X_inc[trn_idx], y_inc[trn_idx]
    val_x, val_y = X_inc[val_idx], y_inc[val_idx]
    
    print('모델 생성')
    lr_clf = LGBMClassifier(learning_rate = 0.005, max_depth  = 40, n_estimators  = 15000,
                            num_leaves = 512,
                           feature_fraction = 0.9,
                           bagging_fraction = 0.7,      
                           seed = seed)  
    
    print('모델 생성 후 학습 시작')
    lr_clf.fit(
            trn_x, trn_y,
            eval_set = [(val_x, val_y)],
            eval_metric = 'logloss',
            verbose =100, 
            early_stopping_rounds = 30
        )
    
    p_tst += lr_clf.predict_proba(X_tst_inc) / folds.n_splits
    p[val_idx, :] = lr_clf.predict_proba(val_x)
    

In [ ]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

In [ ]:
np.savetxt(p_val_file, p, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

#### 제출 파일 생성

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

In [ ]:
sub[sub.columns] = p_tst
sub.head()

In [ ]:
sub.to_csv(sub_file)

# Stacking Model

### 라이브러리 로드

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

import warnings
from pathlib import Path

from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold

import lightgbm as lgbm

In [3]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

### 학습데이터 로드

In [4]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 2020

In [5]:
algorithm_name = 'lgbm'

feature_names= ['stacking-layer1-all']

feature_target_file = feature_dir / f'feature_target.csv'

model_names = []
for feature_name in feature_names:
    model_names.append(f'{algorithm_name}_{feature_name}')
    
stacking_oof_pred_files=[]
for model_name in model_names:
    stacking_oof_pred_files.append( val_dir / f'{model_name}_oof_pred.csv')
    
stacking_test_pred_files=[]
for model_name in model_names:
    stacking_test_pred_files.append( tst_dir / f'{model_name}_test_pred.csv')
    
stacking_submission_files=[]
for model_name in model_names:
    stacking_submission_files.append( sub_dir / f'{model_name}_submission.csv')

### Stacking feature 생성

In [6]:
def load_feature(model_names, number_of_ver=None, kind=None):
    oof_list = []
    test_list = []
    
    if number_of_ver==None or kind==None:
        print('error')
        return None
    
    # 딥러닝 시리즈 4가지 버전
    if kind == 0:
        for model in model_names:
            print(f'load {model}_cv')
            for i in range(1,number_of_ver+1):
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv', delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
    
    # 로지스틱 회귀 6가지 버전
    elif kind == 1:
        for model in model_names:
            print(f'load {model}_cv')
            for i in range(1, number_of_ver+1):
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv', delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))

    # 신경망 기반 불용어 처리 21가지 버전 또는 머신러닝 기반 불용어 처리 18가지 버전
    elif kind == 2:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(2,5):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
    
    # 신경만 기반 불용어 처리 X 13가지 버전 또는 머신러닝 기반 불용어 처리 X 18가지 버전
    elif kind == 3:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(1,2):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
                
    # 첫번째 레이어를 학습하기 위한 데이터셋 모두 가져오기 버전
    elif kind == 4:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(1,5):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
    
    # 두번째 레이어를 학습하기 위한 데이터셋 모두 가져오기 버전
    elif kind == 5:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('stacking') != -1:
                for feature in ['stopwords-yes-nn','stopwords-no-nn','stopwords-no-ml', 'stopwords-no-ml'] :
                    oof_list.append(np.loadtxt(val_dir / f'{model}-{feature}_oof_pred.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}-{feature}_test_pred.csv',delimiter=','))
            elif model.find('all') != -1:
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv',delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred.csv',delimiter=','))
    
    # 데이터셋 + xgb/lgbm tfidf 데이터셋 가져오기 버전 - 하나씩
    elif kind ==6:
        for model in model_names:
            print(f'load {model}_cv')
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv', delimiter=','))
            test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred.csv', delimiter=','))      
    
    return oof_list, test_list

In [7]:
model_names = ['mlp_tfidf-pca', 'lr_tfidf-pca', 'cnn_tfidf-pca', 'lgbm_tfidf','lgbm_tfidf-pca']

tmp_oof, tmp_test = load_feature(model_names, -1, 6)
tmp_oof = np.concatenate(tmp_oof, axis=1)
tmp_test = np.concatenate(tmp_test, axis=1)
print(f'shape : {tmp_oof.shape}, {tmp_test.shape}')

load mlp_tfidf-pca_cv
load lr_tfidf-pca_cv
load cnn_tfidf-pca_cv
load lgbm_tfidf_cv
load lgbm_tfidf-pca_cv
shape : (54879, 25), (19617, 25)


In [8]:
model_names = ['cnn_bow','cnn_feature','cnn_hashing','cnn_tfidf','lr_bow','lr_hashing','lr_tfidf','lstm_feature',
              'mlp_bow','mlp_feature','mlp_hashing','mlp_tfidf','transformer_feature','transformer_v2_feature']

all_oof, all_test = load_feature(model_names, -1, 4)
all_oof = np.concatenate(all_oof, axis=1)
all_test = np.concatenate(all_test, axis=1)
print(f'nn_yes shape : {all_oof.shape}, {all_test.shape}')

load cnn_bow_cv
load cnn_feature_cv
load cnn_hashing_cv
load cnn_tfidf_cv
load lr_bow_cv
load lr_hashing_cv
load lr_tfidf_cv
load lstm_feature_cv
load mlp_bow_cv
load mlp_feature_cv
load mlp_hashing_cv
load mlp_tfidf_cv
load transformer_feature_cv
load transformer_v2_feature_cv
nn_yes shape : (54879, 370), (19617, 370)


In [9]:
all_oof = np.concatenate([all_oof, tmp_oof], axis=1)
all_test = np.concatenate([all_test, tmp_test], axis=1)
print(f'shape : {all_oof.shape}, {all_test.shape}')

shape : (54879, 395), (19617, 395)


In [10]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['index',target_col]).values.flatten()
y.shape

(54879,)

### 스태킹

- 각 oof 마다 fold별로 logloos 변동이 있으므로 최대한 정보를 뽑아내고자 스태킹을 함.

In [11]:
# lgbm
lgbm_params ={
 'bagging_fraction': 0.5265907516925454,
 'bagging_freq': 1,
 'boosting': 'gbdt',
 'class_weight': None,
 'feature_fraction': 0.5915484790928123,
 'lambda_l1': 1.9530422712711874e-05,
 'lambda_l2': 0,
 'learning_rate': 0.014716023892584766,
 'max_depth': 29,
 'metric': 'multi_logloss',
 'min_child_weight': 3.8543859964210974,
 'min_data_in_leaf': 391,
 'n_estimators': 1000,
 'n_jobs': -1,
 'num_class': 5,
 'num_leaves': 105,
 'objective': 'multiclass',
 'random_state': 2020,
 'subsample_for_bin': 160000,
 'verbosity': 0
}

 # bagging_fraction 파라미터가 설정되어 있지만, 실제로는 사용하지 않았음...
 # bagging_freq = 1 로 설정해야지 사용이 가능한데...
 # bagging_fraction을 사용하지 않고, 93.835 라는 결과를 낸 것임.
 # hyperopt를 다시 돌려서 튜닝을 해야 할듯...
 # 그리고 그때 나온 결과가 어떻게 되냐에 따라서 어떤 파라미터를 사용할지 결정을 해야 할듯..
 

In [12]:
datasets = [(all_oof, all_test, y)]

mlogloss = []

lgbm_oof_preds = []
lgbm_test_preds = []

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

for number, (X, test , y) in enumerate(datasets, 1):
    print(f'start : {number}')
    
    lgbm_oof_pred = np.zeros((X.shape[0], n_class))
    lgbm_test_pred = np.zeros((test.shape[0], n_class))
    
    for i, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'training model for CV #{i}')
        
        X_train , X_val = X[i_trn], X[i_val]
        y_train, y_val = y[i_trn], y[i_val]
        
        dtrain = lgbm.Dataset(X_train, label=y_train)
        dval = lgbm.Dataset(X_val, label=y_val)
        watchlist = [(dtrain, 'train'), (dval, 'val')]
        
        clf = lgbm.train(params=lgbm_params, train_set=dtrain, num_boost_round=5000,
                         valid_sets=[dtrain,dval], early_stopping_rounds=50, verbose_eval=5000)
        
        lgbm_oof_pred[i_val, :] = clf.predict(X_val)
        lgbm_test_pred += clf.predict(test) / n_fold
        mlogloss.append(clf.best_score['valid_1']['multi_logloss'])
    lgbm_oof_preds.append(lgbm_oof_pred)
    lgbm_test_preds.append(lgbm_test_pred)
    
    print(f'end : {number}')

start : 1
training model for CV #1
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.093962 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

end : 1


In [13]:
for i,j in enumerate(lgbm_oof_preds,1):
    print(f'logloss = {log_loss(pd.get_dummies(y),j):8.4f}')
    print(f'accuracy = {accuracy_score(y, np.argmax(j,axis=1))*100:8.4f}')
print('mean logloss = ',np.mean(mlogloss))

logloss =   0.3904
accuracy =  85.9163
mean logloss =  0.3903828351640911


### 제출 파일 및 기타 파일 생성

In [14]:
# submission 파일 생성
sub = pd.read_csv(sample_file,index_col=0)

for filename, test_pred in zip(stacking_submission_files, lgbm_test_preds):
    sub[sub.columns] = test_pred
    sub.to_csv(filename)